In [1]:
#!pip install pypdb
import requests
import os
import string
from collections import defaultdict, OrderedDict
from collections import Counter
import re
import shutil
import statistics
from datetime import date
#import hail as hl
import glob
import time
import pytrimal
# Import from installed package
#from pypdb.clients.pdb.pdb_client import *
import dask.dataframe as dd
import json
import pandas as pd
#import plotly as px
from Bio.Seq import Seq
from Bio import SeqIO, Align, PDB, Seq, AlignIO
from Bio.PDB import PDBParser, PDBIO, Select, MMCIFParser, Structure, Chain, Atom
from Bio.PDB import Model as Bio_Model
from Bio.PDB import Chain as Bio_chain
from Bio.SeqIO import PirIO
from Bio.SeqRecord import SeqRecord
from Bio.Align import substitution_matrices
#from Bio import pairwise2
from io import StringIO
from modeller import *
from modeller.automodel import *
from modeller.parallel import job, local_slave
import matplotlib.pyplot as plt
import matplotlib.style as style
import logging
import subprocess
import shlex
from subprocess import PIPE, run
import numpy as np
import math
import seaborn as sns
from concurrent.futures import ThreadPoolExecutor, wait
from functools import partial
from bs4 import BeautifulSoup  #required later to download SIFT files.
import atomium
from itertools import compress
from sklearn import metrics
from sklearn.cluster import AffinityPropagation
from sklearn.datasets import make_blobs
from sklearn.cluster import MeanShift
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.gridspec as gridspe
import plotly.express as px
import plotly.io as pio
from bravado.client import SwaggerClient
from pycanal import Canal
#import hdbscan
from sklearn.cluster import DBSCAN
from pathlib import Path
import concurrent.futures
import threading
from threading import Lock
from concurrent.futures import ProcessPoolExecutor, as_completed
from packman import molecule
from packman.apps import predict_hinge

from sklearn.cluster import OPTICS, cluster_optics_dbscan

#logging.getLogger("requests").setLevel(logging.WARNING)


In [2]:
class DownloadPipe:
    '''Class object containing the download function that will download all pdbs 
    which we need for downstream analysis of a particular uniprot ID'''

    def __init__(self, templates, work_dir, script_dir, seq_id=None, download_type="pdb"):
        self.work_dir = work_dir # Storage of seq identity useful later for temp selection.
        self.script_dir = script_dir #here we store all scripts
        self.seq_id = seq_id
        self.download_type = download_type # Download PDB or also mmCIF (currently only PDB)
        self.pdbs_to_download = templates 
        # The bash script location which will download the pdbs. 
        self.download_script = os.path.join(script_dir, "batch_download_modified.sh") #modify for script location
        self.download_tmp = os.path.join(work_dir, "pdb_list.csv") # The location for the temporary file that is required for the download_script as input.
        self.log_dir = os.path.join(work_dir, "log_files")
        # The list of chains that will be used later to fetch correct structures.
        self.chain_seqid_dict = self._setup_download_list()
        self.temp_seqid_dict = {template: seq_id for template, seq_id in zip(self.pdbs_to_download, self.seq_id)}
        # We store also meta info as a json dict
        self.meta_dict = None
        #we store high resolution structures as a list if the user wants to separate based on resolution.
        self.high_resolution = None
        # set a flag that stops redownloading.
        self.already_downloaded = None
        # collect conservation
        self.conservation_df = None
        #filtered structures based on meta resolution
        self.filtered_structures = None
        #store shifts.
        self.shift_dict = None
            
    def paralellized_download(self):
        '''
        This function is going to call _download_files n times to parallelize download. 
        It is going to pass the function call itself **_download_file**,
        self.download_tmp (the location of the tmp file which is pdb_id comma separated), 
        p (an additional parameter specifying that 
        we want to download pdbs, and self.work_dir(the current work dir)
        '''
        
        self.already_downloaded = self._check_for_pdbs_present()
        # ThreadPoolExecutor
        if self.already_downloaded == False:
            print("we start downloading now:")
            with ThreadPoolExecutor() as executor:
                # Submit your tasks to the executor.
                futures_pdb = [executor.submit(self._download_files, self.download_tmp, 'p', self.work_dir)]
                # Optionally, you can use as_completed to wait for and retrieve completed results.
                for future in as_completed(futures_pdb):
                    result = future.result()
            self.already_downloaded = True
        else:
            print("we already have pdbs from the templates downloaded")
    
    def _setup_download_list(self):
        '''Helper function to setup the list of comma-separated pdb
        ids for the download_files function'''
        chain_seqid_dict = defaultdict(list)
        for pdb, seq_id in zip(self.pdbs_to_download, self.seq_id):
            pdb_4_digit_id = pdb[:4] # e.g 4CFR
            chain = pdb[-1] # e.g A
            chain_seqid_dict[pdb_4_digit_id].append((chain, seq_id)) #map chains and seq id to pdb id
        # We only want to download pdb files once. 
        # No reason to download a PDB-file 4 times just because we need chain [A, B, C, D]
        unique_pdbs = chain_seqid_dict.keys() # Keys : PDB-IDs, Vals: Chains, seq_id
        # Create download_files input list
        with open(self.download_tmp, "w") as pdb_tar:
            pdb_tar.write(",".join(unique_pdbs))
            
            
        #return dict {key: pdb_id = [(chain, seq_id)]}
        return chain_seqid_dict
    
    def _download_files(self, download_tmp, download_type, path)->list:
        """This helper function runs inside paralellized_download 
        and will be used to get the PDB files that we require for downstream analysis."""
        results = []
        # Input for subprocess
        bash_curl_cmd = f"{self.download_script} -f {download_tmp} -o {path} -{download_type}"
        # split into list 
        bash_curl_cmd_rdy = bash_curl_cmd.split()
        
        try:
            # Run subprocess
            result = run(bash_curl_cmd_rdy, stdout=PIPE, stderr=PIPE, universal_newlines=True)
            # Append result output to results
            results.append(result.stdout.split("\n")[:-1])  # Skip the last empty element
        except Exception as e:
            results.append(f"Error downloading: {e}")

        return results    

    def _check_for_pdbs_present(self):
        '''
        Could be good to improve so that if we miss SOME structures we fetch them as well and download ONLY those.
        For those structures we also need seqid per chain and then also update the seqid_chain dict for the whole directory after
        successful download.
        Currently we only check if pdbs are present and if yes we dont download anything further.
        '''
        pdbs_to_retrieve = {f[:4] for f in os.listdir(self.work_dir) if f.endswith(".pdb")}  # Use a set for efficient lookups
        template_codes = {f[:4] for f in self.pdbs_to_download}  # Convert list to set for efficient intersection operation

        # Check for any overlap between the two sets
        overlap = pdbs_to_retrieve.intersection(template_codes)

        print(f"This is overlap in the directory: {overlap}")
        # Return 1 if there is an overlap, else 0
        return True if overlap else False

    
    def retrieve_meta(self, dict_location=None, human_readable=True)->dict:
        '''
        We also want to store meta information about resolution etc.
        This function takes each pdb file and retrieves the following information:
        - Title
        - Keywords
        - PDBcode
        - Authors
        - Deposition date
        - Technique
        - Resolution
        - R_value : If crystallography else None
        - R_free : If crystallographe else None
        - Classification
        - Organism
        - Expression System
        - Number of amino acids in the asymmetric unit
        - Mass of amino acids in the asymmetric unit (Da)
        - Number of amino acids in the biological unit
        - Mass of amino acids in the biological unit (Da)
        '''
        
        json_file_path = os.path.join(self.log_dir, 'meta_dictionary.json')

        for path in [json_file_path, dict_location]: #check first the supposed location alternatively the user supplied location.
            if path and os.path.exists(path):
                try:
                    with open(path, 'r') as json_fh:
                        self.meta_dict = json.load(json_fh)
                except Exception as e:
                    print(f"Error reading {path}: {e}")
                
        #little helper function to deal with date data
        def _date_encoder(obj):
            if isinstance(obj, date):
                return obj.isoformat()  # Convert date to ISO format

        #grab all PDB files which contain the meta information.
        pdbs_to_retrieve = [f for f in os.listdir(self.work_dir) if f.endswith(".pdb")]
        #here we store info about ALL pdbs.
        meta_dictionary = dict()
        
        for pdbs in pdbs_to_retrieve:
            if len(pdbs) == 8: #lets exclude preprocessed pdbs that are longer or shorter.
                sub_dict = dict()
                pdb_code = pdbs[:4]
                try:
                    fullp = os.path.join(self.work_dir, pdbs)
                    pdb = atomium.open(fullp)
                    sub_dict["title"] = pdb.title
                    sub_dict["key_words"] = pdb.keywords
                    sub_dict["code"] = pdb.code
                    sub_dict["authors"] = pdb.authors
                    #sub_dict["deposition_date"] = pdb.deposition_date.isoformat()  #isoformat because it is a time object
                    sub_dict["technique"] = pdb.technique
                    sub_dict["resolution"] = pdb.resolution
                    sub_dict["r_val"] = pdb.rvalue
                    sub_dict["r_free"] = pdb.rfree
                    sub_dict["classification"] = pdb.classification
                    sub_dict["organism"] = pdb.source_organism
                    sub_dict["expression_system"] = pdb.expression_system
                    sub_dict['number_of_residues_asymmetric_unit'] = len(pdb.model.residues())
                    sub_dict['mass_dalton_asymetric_unit'] = f"{pdb.model.mass:.2f}" 
                    try:
                        assembly = pdb.generate_assembly(1) #build the biological assembly 
                        sub_dict['number_of_residues_biological_unit'] = len(assembly.residues())
                        sub_dict['mass_dalton_biological_unit'] = f"{assembly.mass:.2f}"
                    except Exception as e:
                        print(f"We could not build the assembly for: {pdb_code}")
    
                except Exception as e:
                    print(f"We had an error with file: {pdb_code}")
                # store meta info and return
                meta_dictionary[pdb_code] = sub_dict


        #lets store meta info as json dict
        self.meta_dict = meta_dictionary
        
        # Code block to store meta info as a txt file.
        self._save_meta_dict(self.meta_dict, human_readable=human_readable)


    def _save_meta_dict(self, meta_dictionary, human_readable=True):
        '''Helper function to store meta info as a txt file.'''
        #check if log file dir exists, else make it.
        
        if self.log_dir and not os.path.exists(self.log_dir):
            os.makedirs(self.log_dir)

        #lets store the dict in json to read it in for later useage.
        json_file_path = os.path.join(self.log_dir, 'meta_dictionary.json')
        #convert defaultdict to normal dict.
        
        with open(json_file_path, 'w') as json_fh:
            json.dump(meta_dictionary, json_fh, indent=4, default=str)  # Use default=str to handle non-serializable objects

    
    def conservation(self, uniprot_id):
        '''Gets 3 different types of Conservation:
        - Shannon conservation: 
        Shannon entropy. 
        Higher values indicate lower conservation and greater variability at the site.
        
        - Relative conservation:
        Kullback-Leibler divergence.
        Higher values indicate greater conservation and lower variability at the site.
        
        - Lockless conservation
        Evolutionary conservation parameter defined by Lockless and Ranganathan (1999). 
        Higher values indicate greater conservation and lower variability at the site.
        '''

        if self.log_dir and not os.path.exists(self.log_dir):
            os.makedirs(self.log_dir)
        
        mmseq_fasta_result = self._mmseq_multi_fasta(uniprot_id=uniprot_id, outdir=self.work_dir)
        #get 3 different conservation scores in a pandas df.
        conserv_df = self._get_conservation(path_to_msa=mmseq_fasta_result)
        self.conservation_df = conserv_df

        conserv_df.to_csv(f"{self.log_dir}/conservation_df.csv")
        
    def _mmseq_multi_fasta(self, uniprot_id:str, outdir:str, 
                      sensitivity=7, filter_msa=0,
                     query_id = 0.6):
        """
        uniprot_id: The unique uniprot identifier used to fetch the corresponding fasta file that will be used as a template for mmseq2
        outdir: location where result files will be stored.
        sensitivity: mmseq2 specific parameter that goes from 1-7. The higher the more sensitive the search.
        filter_msa = 0 default. if 1 hits are stricter.
        query_id = 0.6 [0, 1]  the higher the more identity with query is retrieved. 1 means ONLY the query hits while 0 means take everything possible.
        """

        #we blast with this fasta as query.
        trgt_fasta_seq = self._get_gene_fasta(uniprot_id)
        #Make outdir for all required files.
        #we need to write it out to file.
        with open(f"{self.work_dir}/{uniprot_id}_fasta.fa", "w") as fasta_out:
            fasta_out.write(f">{uniprot_id}\n")
            fasta_out.write(trgt_fasta_seq)

        #fetch pre downloaded database from a parent folder.
        msa_file = None
        new_location = None
        try:
            DB_storage_location = f"{work_dir}"
            #shutil.copy(previous_path, savepath)
            bash_curl_cmd = f"mmseqs createdb {self.work_dir}/{uniprot_id}_fasta.fa {DB_storage_location}/query_fastaDB" 
            bash_curl_cmd_rdy = bash_curl_cmd.split()
            #run first cmd which setups query database based on our input fasta file
            result_setup_query_db = run(bash_curl_cmd_rdy, stdout=PIPE, stderr=PIPE, 
                                 universal_newlines=True)
            bash_curl_cmd_2 = f"mmseqs search {DB_storage_location}/query_fastaDB {DB_storage_location}/swiss_DB {DB_storage_location}/result_DB {DB_storage_location}/tmp -s {sensitivity}"    
            bash_curl_cmd_rdy_2 = bash_curl_cmd_2.split()
            #run 2nd cmd which blasts against swiss_DB and generates the resultDB (i.e our hits that were found)
            result_setup_blast_db = run(bash_curl_cmd_rdy_2, stdout=PIPE, stderr=PIPE, 
                                 universal_newlines=True)
            #mmseqs convert2fasta DB_clu_rep DB_clu_rep.fasta
            bash_curl_cmd_5 = f"mmseqs result2msa {DB_storage_location}/query_fastaDB {DB_storage_location}/swiss_DB {DB_storage_location}/result_DB {DB_storage_location}/{uniprot_id}_out.fasta --msa-format-mode 3 --filter-msa {filter_msa} --qid {query_id}" 
            bash_curl_cmd_5_rdy = bash_curl_cmd_5.split()
            result_setup_msa_convert = run(bash_curl_cmd_5_rdy, stdout=PIPE, stderr=PIPE, 
                                 universal_newlines=True)
            #delete last line.. required.
            sed_cmd = f'sed -e 1,4d -e $d {DB_storage_location}/{uniprot_id}_out.fasta'        
            bash_curl_cmd_6_rdy = sed_cmd.split()
            #f"{DB_storage_location}/{uniprot_id}_new_out.fasta"
            with open(f"{DB_storage_location}/{uniprot_id}_new_out.fasta", "w") as new_fasta:
                result_truncation = run(bash_curl_cmd_6_rdy, stdout=new_fasta, stderr=PIPE, 
                                 universal_newlines=True)
            # Specify the path to your MSA file
            msa_file = f"{DB_storage_location}/{uniprot_id}_new_out.fasta"
            #transfer the meta file to another location and delete useless files.
            # we need to delete : all uniprot* files. 
            # all query*. All result* 
            new_location = f"{self.work_dir}/{uniprot_id}.fasta"
            shutil.copy(msa_file, new_location)
            #remove_files_and_dirs_msa(DB_storage_location, uniprot_id=uniprot_id)
            
        except Exception as error:
            print(error)
        #we want the path to msa_file for downstream analysis.
        return new_location

    def _get_gene_fasta(self, uniprot_id:str):
        '''
        Helper function to grab the sequence 
        based on the Uniprot ID
        '''
        fields = "sequence"
        URL = f"https://rest.uniprot.org/uniprotkb/search?format=fasta&fields={fields}&query={uniprot_id}"
        resp = self._get_url(URL)
        resp = resp.iter_lines(decode_unicode=True)
        seq = ""
        i = 0
        for lines in resp:
            if i > 0:
                seq += lines
            i += 1
        return seq

    def _get_conservation(self, path_to_msa:str):    
        '''
        Helper function to compute 3 different types of conservation.
        
        - Shannon conservation: 
        Shannon entropy. 
        Higher values indicate lower conservation and greater variability at the site.
        
        - Relative conservation:
        Kullback-Leibler divergence.
        Higher values indicate greater conservation and lower variability at the site.
        
        - Lockless conservation
        Evolutionary conservation parameter defined by Lockless and Ranganathan (1999). 
        Higher values indicate greater conservation and lower variability at the site.
        '''
        canal = Canal(fastafile=path_to_msa, #Multiple sequence alignment (MSA) of homologous sequences
          ref=0, #Position of reference sequence in MSA, use first sequence always
          startcount=0, # ALways 0 because our seqs are always from 1 - end
          verbose=False) # no verbosity 
    
        result_cons = canal.analysis(method="all")
        return result_cons

    def _get_url(self, url):
        '''Helper function that uses requests for Downloads.'''
        try:
            response = requests.get(url)  
            if not response.ok:
                print(response.text)
        except:
            response.raise_for_status()
            #sys.exit() 
        return response
    
    def setup_cutoff(self, cutoff=10, apply_filter=False):
        '''If we want to setup a resolution cutoff filter for further downstream analysis, 
        this function helps with it.'''
        # If there is no meta dict we cant proceed and filter based on resolution.
        if self.meta_dict:
            #here we store the pdb codes that we keep
            pdbs_to_keep = []
            #Now lets parse through the whole meta dict and fetch the cutoffs for structures.
            for _, single_pdbs in self.meta_dict.items():
                if single_pdbs['resolution'] <= cutoff:
                    pdbs_to_keep.append(single_pdbs['code'].lower()) #normalize to lower in order to have uniform list members.   
                    
            self.filtered_structures = pdbs_to_keep
            #now if we directly want to apply the filter to remove files that dont match our criteria.
            if apply_filter:
                #check for union between files and kept structures.
                pdbs_to_retrieve = [f[:4] for f in os.listdir(self.work_dir) if f.endswith(".pdb")]
                #lets fetch the intersect between the 2 sets which corresponds to the pdbs we want to keep.
                common_pdb = set(pdbs_to_retrieve) & set(pdbs_to_keep) #intersection
                intersect_lst = list(common_pdb)
                self.filtered_structures = intersect_lst
                if self.chain_seqid_dict:
                    #now we need to update the chain_dict as well:
                    filtered_dict = {pdb: v for pdb, v in self.chain_seqid_dict.items() if pdb[:4] in self.filtered_structures}
                    self.filtered_structures = filtered_dict
                    
        else:
            print("We have no meta dict to implement a cutoff")
            #In this case we take all.
            pdbs_to_retrieve = [f[:4] for f in os.listdir(self.work_dir) if f.endswith(".pdb") and len(f) == 8] #exclude non original files. Only store pdb + _ + chains.
            self.filtered_structures = pdbs_to_retrieve

    def parallel_shift_calculation(self):
        '''Here we compute the shift according to uniprot or authors
        in order to be in line with UNIPROT numbering which is crucial for later renumbering.'''
        
        pdbs_to_retrieve = [f[0:4] for f in os.listdir(self.work_dir) if f.endswith(".pdb")]  
        pdbs_to_retrieve = set(pdbs_to_retrieve) & set(x[:4] for x in self.oligodict.keys()) #here we check the first 4 which is pdb code
        link_path = "https://www.ebi.ac.uk/pdbe/api/mappings/uniprot"
        shift_dict = defaultdict()
        
        with ThreadPoolExecutor() as executor:
            calculate_shift_bound = partial(self._calculate_shift)
            tasks = ((link_path, pdb) for pdb in pdbs_to_retrieve)
            # Map the bound function to the arguments in parallel
            results = executor.map(calculate_shift_bound, tasks)
            for result in results:
                for keys, vals in result.items():
                    shift_dict[keys] = vals
                    
        self.shifts = shift_dict

    def _calculate_shift(self, args):
        '''
        Helper function to compute the shift.
        Args: link_path to UNIPROT page and the pdb path.
        '''
        link_path, pdb = args
        shift_dict = defaultdict()
        searchp = f"{link_path}/{pdb[0:4]}"
        resp = self._get_url(searchp)
        resp = resp.json()
        for pdb_id, pdb_info in resp.items():
            for uniprot_id, uniprot_info in pdb_info['UniProt'].items():
                for mapping in uniprot_info['mappings']:
                    chain_id = mapping['chain_id']
                    unp_start = mapping['unp_start']
                    unp_end = mapping['unp_end']
                    author_start = mapping['start']['author_residue_number']
                    author_end = mapping['end']['author_residue_number']
    
                    if author_start is None:
                        author_start = unp_start
                    if author_end is None:
                        author_end = unp_end
                    shift_start = unp_start - author_start
                    shift_end = unp_end - author_end
                    shift_dict[f"{pdb_id}_{chain_id}"] = shift_start 
                    
        self.shift_dict = shift_dict
        return shift_dict

    
    def parallel_renumbering(self):
        '''
        Helper function to do parallelized renumbering.
        If already renumbered, don't do it again.
        '''
        if self.renumbered:
            print("You already renumbered your structures based on shift.")
            return  # Exit the function early

        if not self.shifts:
            print("You first need to obtain shifts which will be used as reference in order to start renumbering.\nCall .parallel_shift_calculation() first.")
            return  # Exit the function if no shifts are available

        # At this point, we know renumbering has not been done and shifts are available
        relevant_files = self.chain_seq_dict.keys()
        with ProcessPoolExecutor() as executor:
            # Using partial to create a function with fixed parameters (shift_dict, path)
            renumber_structure_partial = partial(self._renumber_structure, shift_dict=self.shifts, path=self.work_dir)
            # Map the renumbering function to each relevant file in parallel
            executor.map(renumber_structure_partial, relevant_files)

        self.renumbered = True

    
    def _renumber_structure(self, files, shift_dict, path):
        '''Function that is going to apply pdb_shiftres_by_chain.py to each pdb file that is shifted.
        Will apply renumbering to ALL structures if you did not set a cutoff previously and applied filter. 
        If filter applied for resolution will only renumber those structures that are left after filtering.'''
        for keys, vals in shift_dict.items():
            #dont renumber if there is not shift
            if files == keys[0:4] and vals != str(0):
                chain = keys[-1]
                shift = int(vals)
                filepath = f"{self.work_dir}/{files}.pdb"
                # Should we really shift by shift + 1??? or just shift?
                bash_cmd = f"python {self.script_dir}/pdb_shiftres_by_chain.py {filepath} {shift} {chain}"
                bash_cmd_rdy = bash_cmd.split()
            
                with open(f"{filepath}_tmp", "w") as fh_tmp:
                    result = run(bash_cmd_rdy, stdout=fh_tmp, stderr=PIPE, universal_newlines=True)
                    # Now replace the original one with the temp file.
                    os.replace(f"{filepath}_tmp", filepath)

This is overlap in the directory: {'7nnj', '5ltu', '3mcf', '2duk', '7tn4'}
we already have pdbs from the templates downloaded


In [4]:
class PDBBuilder:
    '''Class to build biological units from asymmetric units'''
    def __init__(self, work_dir, structures, remove_intermediates=False, main_protein_sequence=None, logging=True):
        self.work_dir = work_dir
        #Here we store the structures that need to be built.
        self.structures = structures
        #we store intermediate files per default.
        self.remove_intermediates = remove_intermediates
        #we set oligo to None but store later the oligodict
        self.oligodict = None
        # dict to store the potential ranges.
        self.range_dict = None
        # logging purpose
        self.log_dir = os.path.join(work_dir, "log_files")
        # enable logging.
        self.logging = logging
        #main_prot_seq
        self.main_protein_sequence = main_protein_sequence
        # result dict for established domains
        self.established_domain_dict = None
        # top templates found for each range
        self.top_templates = None

        self.assemblied_structures = None

    def build_assembly(self):
        '''
        This function takes all pdbs in self.structures that were passed to class initialization:

        structure:
        self.structures -> dict
        keys: pdb_id_4_digit
        values: tuple (chain_id, seq_identity)

        example: {'5ltu': [('A', 0.96), ('B', 0.96)]}

        We rechain all pdbs in order to have uniformized chain labels for downstream processing.
        Then, we build biological assemblies based on the asymmetric unit deposition and caputure the oligomeric status.
        We return then an updated self.oligodict that has the following structure:

        key: PDB_ID_CHAIN(s)
        values: tuple(chain, seq_id)
        example: {'5ltu_AB.pdb': [('A', 0.96), ('B', 0.96)]}
        This corresponds to the fully assembled structure.
        
        '''
        # These files need to be opened, rechained and assemblies built.

        full_pdb_paths = [os.path.join(self.work_dir, f"{file}.pdb") for file in self.structures]
        oligostates = defaultdict(str)
        #this letterdict is used for rechaining.
        letterdict = {i: chr(65 + i) for i in range(26)}
        #changed this here from threadpool to process pool
        with ProcessPoolExecutor() as executor:
            # Define your processing function, partially applied with gene_name and main_protein_seq
            process_func = partial(self._process_pdb, letterdict=letterdict)
            results = executor.map(process_func, full_pdb_paths)
            for result in results:
                #print(f"this is result: {result}")
                oligostates.update(result)

        #beautifully chaotic.... basically we update... 4 digit ID + chains i.e 4 means ABCD 2 means AB 1 means A, and concatenate it with .pdb to get 4rt4_ABCD.pdb
        new_oligostates = {f"{k[0:4]}_{''.join(letterdict[i] for i in range(v))}.pdb": v for k, v in oligostates.items()}

        #lets update the dict now with info about seq id.
        self._update_oligostates_with_sequence_identities(new_oligostates)

    def _update_oligostates_with_sequence_identities(self, dict_to_update):
        '''
        This function helps to bring back the sequence identity information to the new oligodict
        For those chains that wont end up in the biological unit, we drop the information.
        '''
        updated_oligostates = {}
        for pdb_chain_file, oligostate in dict_to_update.items():
            pdb_id = pdb_chain_file.split('_')[0]  # Extract the PDB ID from the filename
            chain_ids = pdb_chain_file.split('_')[1].rstrip('.pdb')  # Extract chain IDs
            # Initialize a list to hold sequence identities for each chain
            seq_identities = []
            
            if pdb_id in self.structures:
                # Iterate through each chain and its corresponding sequence identity in the structures
                for chain_id, seq_identity in self.structures[pdb_id]:
                    # If the current chain is part of the oligostate (i.e., it's in the chain_ids string), add its sequence identity
                    if chain_id in chain_ids:
                        seq_identities.append((chain_id, seq_identity))
            
            # Update the dictionary with the new value format
            updated_oligostates[pdb_chain_file] = (seq_identities)
        
        # Update the class attribute
        self.oligodict = updated_oligostates
    
    def _process_pdb(self, path:str,letterdict:dict)->dict:
        #helper function to split between nmr and xray / cryoem
        try:
            pdb_file_name = os.path.basename(path)
            if len(pdb_file_name) != 8:
                #this means its already processed previously because its not 5DUK_A.pdb instead of 5DUK.pdb
                oligostate = len(pdb_file_name[5:-4]) # this is the oligostate e.g ABC = 3 
                return {pdb_file_name: oligostate}
                
            pdb_file = atomium.open(path)
            model_len = len(pdb_file.models)
            
            if model_len > 5:  # if multiple models => NMR OR CRYO-EM?
                return {pdb_file_name: self._NMR_ensemble(path=path, letterdict=letterdict)}
            else:
                return {pdb_file_name: self._non_NMR_structures(path=path, letterdict=letterdict)}

            #now we remove the original file.
            os.remove(path)
            
        except Exception as error:
            print("process pdb did not work")
            print(error)
            return {}
    
    #helper function for XRAY and CRYO-EM ensembles.
    def _non_NMR_structures(self, path:str, letterdict:dict):
    
        """This function takes in the the pdb file that is xray or cryoem and rechains each chain. 
        Additionally, we merge the new labelled chains into a merged_pdb file for further use."""
    
        #store base dir.
        base_dir = os.path.dirname(path)
        pdb_name = os.path.basename(path)[:4]
        #/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUDT4B/3h95.pdb
        pdb_file = atomium.open(path)
        assemblies = [pdb_file.generate_assembly(n + 1) for n in range(len(pdb_file.assemblies))]
        #we take the first one(this is the biological unit built from the asymmetric unit)
        assembly = assemblies[0]
        #for logger purpose
        #assembly_info = f"{pdb_name=}, {assemblies=}"
        #self._logger(assembly_info)        
        #tuple containing chain ID and LEN of each chain.
        seq_chains = [(chain.id, len(chain.sequence)) for chain in assembly.chains()]
        sorted_lens = sorted(seq_chains, key= lambda x: x[1], reverse=True) #reverse = true :largest first.
        accepted_chains = []  #this will be used to store and evalute oligomeric state.
        min_accepted_length = float('inf')   # Minimum length of accepted chains init as pos inf.

        for chain, length in seq_chains:
            #for each chain and length.
            if not accepted_chains or length > 0.8 * min_accepted_length:
                #we accept it if either we have no other chain so far OR length > 80% of the chain we have so far.
                accepted_chains.append(chain)
                #first chain will be accepted and then will be the standard for the next chains to follow.
                min_accepted_length = min(min_accepted_length, length)

        oligostate = len(accepted_chains)  #this excludes small peptides ect from being mistaken as oligomers.
        """Part 2. We investigate oligomeric state."""
        accepted_chains_set = set(accepted_chains)
        oligomeric_status = None
        if len(accepted_chains_set) != len(accepted_chains):
            #this means we have a homo-oligomer!
            #e.g A vs A A A .. len(1) != len(0)
            #hetero-mers are not caught here.. A B C == A B C == len(3)
            oligomeric_status = "homo_oligomer"
        
        elif len(accepted_chains) == 1:
            #this means we deal with a monomer.
            oligomeric_status = "monomer"
        
        elif len(accepted_chains) > 1 and len(accepted_chains) == len(accepted_chains_set):
            #this means its a mixed heteromer. becaue len(1) > AND set == list aka no redundancy ergo heteromer.
            oligomeric_status = "hetero_oligomer"

        """Part 3: We follow through and now save individual chains + send them to proper rechaining. """    
        path_list = []

        for idx, chain in enumerate(assembly.chains()):
            chain_label = chain.id
            if chain_label in accepted_chains_set:
                path_to_pdb = f"{self.work_dir}/{pdb_name}_{idx}.pdb"
                #save it here.
                path_list.append(path_to_pdb)
                #and also save the structure in its wrong chain state first.
                chain.save(path_to_pdb)
        
            path_list = sorted(path_list, key=lambda x: int(x[-5]))
    
        """Part 4: We now deal with all kind of oligomers, and also save all single chains in the procedure.
        Normal monomers are also simply saved and rechained. Everything according to a general schema for efficient
        downstream processing."""
    
        self._merge_pdb_chains(path_list, pdb_name=pdb_name, oligomeric_status=oligomeric_status, 
                      letterdict=letterdict, accepted_chains = accepted_chains, accepted_chains_set=accepted_chains_set)
        
        #we return the oligostate of this file and merge it into dict as return value.
        return oligostate

    #helper function for NMR ensembles.
    def _NMR_ensemble(self, path:str, letterdict:dict):

        """This function takes in the NMR ensemble and 
        splits each state into a respective PDB file."""
    
        #open the pdb file
        print(f"we currently open with atomium: {path}")
        pdb_name = os.path.basename(path)[:4] #4 digit ID
        base_dir = os.path.dirname(path) #base dir name
        pdb_file = atomium.open(path)
        oligostate = 1 #default initialize
        path_list = []
    
        for i, model in enumerate(pdb1.models):
            #now iterate through each model and its respective chains.
            chain_len = len([x.id for x in model.chains()]) > 1 # True if multiple chains.
            #if larger than 1 : we need to merge.
            chain_paths = []
            new_chains = []
            
            for j, chain in enumerate(model.chains()): #enumerate because there are NMR models with MULTIPLE CHAINS
                #here we save the structure. as number.. we need to check the chain id.
                new_chain = chain.copy(id=letterdict[j]) #new chain ID.
                #this effectively rechained the chain.
                save_location = f"{base_dir}/{pdb_name}_{i}_{letterdict[j]}.pdb"
                new_chain.save(save_location) # e.g 4ND5_0_A.pdb 4ND5_1_A etc..
                  
                if chain_len:
                    chain_paths.append(save_location)
                    new_chains.append(new_chain.id)
        
            #if done: check if there are multiple chains. if yes. merge.
            if chain_len:
                oligostate = len(new_chains)
                #we merge the chains.
                save_nmr_oligomer = f"{base_dir}/{pdb_name}_{i}_{''.join(new_chains)}.pdb"
                merge_command = f"python {self.work_dir}/pdb_merge.py {' '.join(chain_paths)}"
                merge_command_rdy = merge_command.split()
                merge_output_file = f"{save_nmr_oligomer}_tmp.pdb"  #tmp
    
                with open(merge_output_file, "w") as fh_out:
                    result_pdbs = run(merge_command_rdy, stdout=fh_out, stderr=PIPE, universal_newlines=True)
        
                # Run tidy on the merged PDB
                tidy_command = f"python {self.work_dir}/pdb_tidy.py {merge_output_file}"
                tidy_command_rdy = tidy_command.split()
                tidy_output_file = f"{save_nmr_oligomer}.pdb"
    
                with open(tidy_output_file, "w") as fh_out2:
                    results_tidy = run(tidy_command_rdy, stdout=fh_out2, stderr=PIPE, universal_newlines=True)

                #we remove tmp intermediate files.
                if self.remove_intermediates:
                    print("we remove", merge_output_file)
                    os.remove(merge_output_file) #this is the tmp file that is not tidy.

        return oligostate
    
    def _merge_pdb_chains(self, path_list:list, pdb_name:str, oligomeric_status:str, letterdict:dict,
                     accepted_chains:list, accepted_chains_set:set):

        if oligomeric_status == "homo_oligomer":
            self._pure_oligomer_rechaining(path_list=path_list, letterdict=letterdict, pdb_name=pdb_name)
            
        elif oligomeric_status == "hetero_oligomer":
            self._mixed_oligomer_rechaining(path_list=path_list, letterdict=letterdict, pdb_name=pdb_name,
                                  accepted_chains=accepted_chains, accepted_chains_set=accepted_chains_set)
            
        elif oligomeric_status == "monomer":
            self._monomeric_rechaining(path_list=path_list, letterdict=letterdict, pdb_name=pdb_name)
            
        else:
            print(f"There was an issue with: {oligomeric_status=}")
            
        return

    def _pure_oligomer_rechaining(self, path_list:list, letterdict:dict, pdb_name:str): 
        #path_list = ['/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUDT4B/3h95_0.pdb',
        # '/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUDT4B/3h95_1.pdb']
        directory = os.path.dirname(path_list[0]) #this does not change. so no reason to constantly evaluate it in the loop
        #store path to chains here.
        lst_to_merge_paths = []
        #store lists here for later merge.
        chain_lst = []

        for path_to_pdb in path_list:
            
            filename = os.path.basename(path_to_pdb) #same as above.
            new_chain_digit = filename[5]  # Get the single chain id (e.g., '0')
            parser = PDBParser(QUIET=True)
            individual_structure = parser.get_structure("default", path_to_pdb)
            new_chain = letterdict[int(new_chain_digit)]

            for models in individual_structure:
                for chain in models:
                    if chain.id == new_chain:
                        #then we simply save it under its original chain.
                        io = PDBIO()
                        io.set_structure(chain)
                        save_location = os.path.join(directory, f"{pdb_name}_{chain.id}.pdb")
                        lst_to_merge_paths.append(save_location)
                        chain_lst.append(chain.id)
                        io.save(save_location)
                    else:
                        chain.id = new_chain
                        io = PDBIO()
                        io.set_structure(chain)
                        save_location = os.path.join(directory, f"{pdb_name}_{chain.id}.pdb")
                        chain_lst.append(chain.id)
                        lst_to_merge_paths.append(save_location)
                        io.save(save_location)
            
            if self.remove_intermediates:
                os.remove(path_to_pdb) #we no longer need the original file

        #prepare subprocess for pdb_merge.py
        merge_command = f"python {self.work_dir}/pdb_merge.py {' '.join(lst_to_merge_paths)}"
        merge_command_rdy = merge_command.split()
        merge_output_file = f"{self.work_dir}/{pdb_name}_{''.join(chain_lst)}_tmp.pdb"  #tmp
    
        with open(merge_output_file, "w") as fh_out:
            result_pdbs = run(merge_command_rdy, stdout=fh_out, stderr=PIPE, universal_newlines=True)
        
        # prepare subprocess for pdb_tidy.py
        tidy_command = f"python {self.work_dir}/pdb_tidy.py {merge_output_file}"
        tidy_command_rdy = tidy_command.split()
        tidy_output_file = f"{self.work_dir}/{pdb_name}_{''.join(chain_lst)}.pdb"
    
        with open(tidy_output_file, "w") as fh_out2:
            results_tidy = run(tidy_command_rdy, stdout=fh_out2, stderr=PIPE, universal_newlines=True)

        os.remove(merge_output_file) #this is the tmp file that is not tidy.
        #lets remove artifacts if the user wishes
        if self.remove_intermediates:
            for artifacts in lst_to_merge_paths:
                os.remove(artifacts)

    def _mixed_oligomer_rechaining(self, accepted_chains:list,
                               accepted_chains_set:set,
                               path_list:list,
                               letterdict:dict, pdb_name:str):

        seen_chains = sorted(accepted_chains, reverse=False)
        chain_seq_len = len(seen_chains) #e.g 6
        shift = len(accepted_chains_set) # e.g 3
        blocksize = chain_seq_len // shift # e.g 2
        block_count = int(chain_seq_len/blocksize) 
        '''
        # A A B B C C becomes A D B E C F
        # B B C C becomes A C B D 
        # i = 1
        # A D 
        # block 1 2 3 for A A B B C C 
        # 0 2 1 3
        '''
        j = 0
        new_chain_seq = []
        lst_to_merge_paths = []
    
        for blocks in range(0, block_count):
            for i in range(0, blocksize):
                '''
                # first iteration A D
                # second iteration B E
                # third iteration C F
                '''
                new_chain = letterdict[blocks+i*shift]
                new_chain_seq.append(new_chain)
                path_to_pdb = path_list[j]
                directory = os.path.dirname(path_to_pdb)
                j += 1
                parser = PDBParser(QUIET=True)
                prot_name = f"default"
                #open the correct pdb and rechain it.
                structure_template = parser.get_structure(prot_name, path_to_pdb)
                
                for models in structure_template:
                    for chain in models:
                        if chain.id != new_chain:
                            chain.id = new_chain
                
                        io = PDBIO()
                        io.set_structure(chain)
                        #print(f"This is single chain save inside oligomer rechain: {directory}/{pdb_name}_{chain.id}.pdb")
                        io.save(f"{directory}/{pdb_name}_{chain.id}.pdb")
                        lst_to_merge_paths.append(f"{directory}/{pdb_name}_{chain.id}.pdb")

                #remove intermediate file
                os.remove(path_to_pdb)

        # Prepare subprocess for pdb_merge.py
        merge_command = f"python {self.work_dir}/pdb_merge.py {' '.join(lst_to_merge_paths)}"
        merge_command_rdy = merge_command.split()
        merge_output_file = f"{self.work_dir}/{pdb_name}_{''.join(new_chain_seq)}_tmp.pdb"  #tmp
        
        with open(merge_output_file, "w") as fh_out:
            result_pdbs = run(merge_command_rdy, stdout=fh_out, stderr=PIPE, universal_newlines=True)
            
        # Prepare subprocess for pdb_tidy.py
        tidy_command = f"python {self.work_dir}/pdb_tidy.py {merge_output_file}"
        tidy_command_rdy = tidy_command.split()
        tidy_output_file = f"{self.work_dir}/{pdb_name}_{''.join(new_chain_seq)}.pdb"
        
        with open(tidy_output_file, "w") as fh_out2:
            results_tidy = run(tidy_command_rdy, stdout=fh_out2, stderr=PIPE, universal_newlines=True)
    
        os.remove(merge_output_file) #this is the tmp file that is not tidy.  
        #lets remove artifacts if the user wishes
        if self.remove_intermediates:
            for artifacts in lst_to_merge_paths:
                os.remove(artifacts)
    
    def _monomeric_rechaining(self, path_list:list,
                          letterdict:dict,
                          pdb_name:str):

        #path_list=['/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUDT4B/2q9p_0.pdb'], 
        #letterdict={0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K',
        #11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21:
        #'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}, 
        
        parser = PDBParser(QUIET=True)
        prot_name = "default"
        #We only have 1 path in this list.
        pdb = path_list[0]
        # Open the correct PDB and rechain it.
        dir_name = os.path.dirname(pdb)
        structure_template = parser.get_structure(prot_name, pdb) 
        # Get the new chain ID
        new_chain = "A"  #always... A
        for model in structure_template:
            for original_chain in model:
                if original_chain.id != new_chain:
                    original_chain.id = new_chain
        save_path = os.path.join(dir_name, f"{pdb_name}_{new_chain}.pdb")
        #print(f"This is save path: {save_path=}")
        # Save the modified structure
        io = PDBIO()
        io.set_structure(structure_template)
        #print("we save now:")
        io.save(save_path)
        if self.remove_intermediates:
            #print("we remove", pdb)
            os.remove(pdb)
    
    def create_domain_boundaries(self, save=True):
        '''
        Process PDB data by getting real ranges and then splitting domains.
        This is the single entry point for users to execute the workflow.
        '''
        
        # Step 1: Get Real Ranges
        self.range_dict = self._get_real_ranges()

        # Step 2: Split Domains
        # Ensure that range_dict is not None or handle the case if it is
        if self.range_dict is not None:
            self.established_domain_dict = self._split_domains_pdb(self.range_dict)
            #logging
            if logging:
                outp = os.path.join(self.log_dir, "established_domains.json")
                converted_dict = self._convert_keys_to_string(self.established_domain_dict)
                self._logging(outp, converted_dict)
            
        else:
            print("Range dictionary is empty. Cannot proceed with splitting domains.")
            return None


    def _get_real_ranges(self):
        '''
        Here we try to find out if the protein of interest is a multi-domain or single domain protein.
        If there are many structures that consist of multiple different ranges that might or might not overlap,
        we will end up with N domains corresponding to K ranges where K can be larger than N (if the domain itself has many different ranges)
        '''
        real_range_dict = defaultdict()
        pdbs = [os.path.join(self.work_dir, k) for k in self.oligodict]

        with ProcessPoolExecutor() as executor:
            results = list(executor.map(self._get_range_parallelized, pdbs))
        # Process the results
        for hit in results:
            for pdb_path, chains, min_num, max_num in hit:
                real_range_dict[pdb_path] = (min_num, max_num)

        return real_range_dict
    
    def _get_range_parallelized(self, pdb_path:str):
        """
        This function gets called by get_real_ranges and leverages parallelization 
        to speed up domain boundary computations
        """
        hits = []
        try:
            parser = PDBParser()
            structure = parser.get_structure("none", pdb_path)
            chains_residues = []
            for model in structure:
                for chain in model:
                    chain_id = chain.id
                    # Filter the residues within the chain
                    residues_in_chain = [res for res in chain if res.get_id()[0] == " "]
                    result = [res.get_id()[1] for res in residues_in_chain]
                    chains_residues.append((chain_id, result))

            #now we have all chains and their ids
            if len(chains_residues) == 1: #means its monomer 
                for chains, resids in chains_residues:
                    min_num, max_num = min(resids), max(resids)
                    #print(pdb_path, min_num, max_num)
                    hits.append((pdb_path, chains, min_num, max_num))
                
            elif len(chains_residues) > 1: # oligomer.
                #real chains is in the name of the pdb.
                #'/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/
                #NUDT4B/merged_cleaned_files/3i7u_ABCD.pdb'
                chain_name = os.path.basename(pdb_path)[5:-4]  #this should be ABCD in the case above
                min_max_list = []
                for chains, resids in chains_residues:
                    chains += chains
                    min_num, max_num = min(resids), max(resids)
                    min_max_list.append((min_num, max_num))

                lowest_start = min([x[0] for x in min_max_list])
                highest_stop = max([x[1] for x in min_max_list])
            
                hits.append((pdb_path, chain_name, lowest_start, highest_stop))
            
        except Exception as e:
            print(e)
        
        return hits

    def _split_domains_pdb(self, range_dict):
        '''
        Function to split domains based on the ranges obtained in the previous step.
        '''
        # lets first do a quick merge
        merged_dict = self._merge_paths_within_interval(range_dict)
        #print(f"This is merged_dict_length: {len(merged_dict.values())}")
        #now we start iterative merging of overlaps.
        domain_dict = self._make_groups_iter_pdb(merged_dict, num_of_iterations=0 , break_point=10)
        #print(f"This is domain_dict_length after make_groups_iter: {len(domain_dict.values())}")
        cleaned_dict = defaultdict()
    
        for ranges, twisted_path_lists in domain_dict.items():
            #now we need to flatten domain_dict
            flattened_data = self._flatten_nested_lists(twisted_path_lists)
            #print(flattened_data)
            # Use a set to remove duplicates
            unique_flattened_data = set(flattened_data)
            unique_flattened_list = list(unique_flattened_data)
            #this works.
            cleaned_dict[ranges] = unique_flattened_list

        prot_length = 10 #len(main_prot_seq) test purpose

        #lets try this first.
        if self.main_protein_sequence: # this case we can handle by fetching the uniprot id and the associated structure.
            if len(self.main_protein_sequence) > 300:
                tolerance = 0.3 * prot_length
            else:
                tolerance = 80
        else:
            tolerance = 80
            
        #print(f"This is cleaned_dict_length before merge_overlapping_int: {len(cleaned_dict.values())}")
        domain_dict = self._merge_overlapping_intervals(cleaned_dict, tolerance=tolerance)
    
        #print(f"This is cleaned_dict_length after merge_overlapping_int: {len(domain_dict.values())}")
        return domain_dict

    def _make_groups_iter_pdb(self, merged_dict:dict, num_of_iterations, break_point):
        #print(f"This is iteration : {num_of_iterations}")
        while True:
            if num_of_iterations > break_point:
                 #that means we cant progress.
                 break
            #print(f"iteration number: {num_of_iterations}")
            if len(merged_dict) == 1:
                return merged_dict
    
            num_of_iterations += 1
            ranges_1 = []
            ranges_2 = []
    
            #attach key ranges to both lists
            for keys, vals in merged_dict.items():
                ranges_1.append(keys)
                ranges_2.append(keys)
        
            union_dict = defaultdict(list)        
            
            for range_to_check in ranges_1:  #lets parse through the ranges.
                union_new = False  #default false for start.
                for range_to_compare in ranges_2: #we compare our hit against all potential ranges.
                    if range_to_check != range_to_compare:  #means they are different.
                        
                        range_to_check_paths = merged_dict[range_to_check]
                        range_to_comp_paths = merged_dict[range_to_compare]
                        
                        range_abs_check = np.abs(int(range_to_check[1])-int(range_to_check[0]))
                        range_abs_compare = np.abs(int(range_to_compare[1])-int(range_to_compare[0]))
    
                        intersect_between_both = self._get_intersect(range_to_check, range_to_compare)# gets abs length of intersect.
                        #range abs check and range abs comp are integers. 
                        #intersect is int and corresponds to length between intersection of both.
                        #cond 1: if the intersection between the 2 ranges is LESS than the absolute length of range to check and 
                        #the intersection is also GREATER than 80% of the first interval.
                        condition_1 = intersect_between_both <= range_abs_check and intersect_between_both >= 0.8* range_abs_check  
                        #cond 2: if the intersection between the 2 ranges is LESS than the absolute length of range to compare and 
                        #the intersection is also GREATER than 80% of the 2nd interval.
                        condition_2 = intersect_between_both <= range_abs_compare and intersect_between_both >= 0.8* range_abs_compare
                        #this means the intersect is less than the original range and the intersect is also larger than 80% of the orignal range.
                        #as well as the same for the second range to compare.
                        if condition_1 and condition_2:
                            union_new = self._merge_into_union(range_to_check, range_to_compare)
    
                if union_new:
                    if union_new in union_dict:
                        union_dict[union_new].append(range_to_check_paths + range_to_comp_paths)
                    else:
                        union_dict[union_new] = range_to_check_paths + range_to_comp_paths
    
                else:
                    union_dict[range_to_check].extend(range_to_check_paths)
                
            merged_dict = union_dict
            merged_dict = self._make_groups_iter_pdb(merged_dict, num_of_iterations, break_point)
    
        return merged_dict

    def _get_intersect(self, range_to_check, range_to_compare):

        # range_to_check = (start, stop)
        # range_to_compare = (start, stop)
        start_check = int(range_to_check[0])
        stop_check = int(range_to_check[1])
    
        start_comp = int(range_to_compare[0])
        stop_comp = int(range_to_compare[1])
        
        full_range_to_check = set([x for x in range(start_check, stop_check+1)])
        full_range_to_compare = set([x for x in range(start_comp, stop_comp+1)])
    
        #now lets grab set for both
        intersection = full_range_to_check.intersection(full_range_to_compare)
        #we are interested in the length of the intersect
        return len(intersection)

    def _merge_dicts_range_seqid(self, range_dict, seqid_dict):
        merged_dict = {}

        #     #{'5ltu_AB.pdb': [('A', 0.96), ('B', 0.96)], is seqid_dict here.
        '''
        range_dict={(1, 151): [('/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/7nnj_A.pdb', 'A'), 
        ('/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/7tn4_A.pdb', 'A'),
        ('/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/3mcf_A.pdb', 'A'),
        ('/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/2duk_A.pdb', 'A'),
        ('/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/5ltu_AB.pdb', 'AB')]}, 
        seqid_dict={'5ltu_AB.pdb': [('A', 0.96), ('B', 0.96)], '7nnj_A.pdb': [('A', 0.956)], 
        '2duk_A.pdb': [('A', 0.967)], '3mcf_A.pdb': [('A', 0.952)], '7tn4_A.pdb': [('A', 0.787)]}
        '''

        for range_key, pdb_entries in range_dict.items():
            merged_list = []
            for pdb_path, chain_group in pdb_entries:
                pdb_file = os.path.basename(pdb_path)  # Extracts '5ltu_AB.pdb' from the full path
                # Check if the PDB file is in the seqid_dict
                if pdb_file in seqid_dict:
                    # Extract sequence identities for the relevant chains
                    for chain_id, seq_id in seqid_dict[pdb_file]:
                        if chain_id in chain_group:
                            # Append the full path for consistency with your range_dict structure
                            merged_list.append((pdb_path, (chain_id, seq_id)))
                else:
                    print(f"Warning: No seqid found for {pdb_file}")
            merged_dict[range_key] = merged_list

        return merged_dict


    def _merge_into_union(self, range_to_check, range_to_compare):

        start_check = int(range_to_check[0])
        stop_check = int(range_to_check[1])
    
        start_comp = int(range_to_compare[0])
        stop_comp = int(range_to_compare[1])
    
        full_range_to_check = set([x for x in range(start_check, stop_check+1)])
        full_range_to_compare = set([x for x in range(start_comp, stop_comp+1)])
    
        union_merge = full_range_to_check.union(full_range_to_compare)
        
        sorted_union = sorted(union_merge)
        result = (sorted_union[0], sorted_union[-1])
        
        return result


    def _merge_overlapping_intervals(self, path_interval_dict, tolerance):

        #input: key: ranges values = list of paths.
        intervals = list(path_interval_dict.keys())
    
        #[(5, 500), (1, 507), (3, 507), (1, 535), (8, 500), (4, 507)]
        #print(f"these are the intervals we deal with in merging: {intervals}")
        merged_intervals = self._merge_intervals(intervals, tolerance)
    
        merged_dict = {}
        #print(f"this is path_interval_dict : {path_interval_dict}")
        for merged_interval in merged_intervals:
            merged_paths = []
                
            for interval, path_chain_list in path_interval_dict.items():
                if self._is_within_tolerance(merged_interval, interval, tolerance):
                    #print(f"{path_chain_list=}")
                    for path, chain in path_chain_list: # list of lists consisting of tuples a (path / chain)
                        merged_paths.append((path, chain))
            merged_dict[merged_interval] = merged_paths
    
        return merged_dict


    def _merge_intervals(self, intervals, tolerance):

        merged = [intervals[0]]  # Initialize with the first interval
        for start, end in intervals[1:]:
            merged_interval = None
    
            for i, (merged_start, merged_end) in enumerate(merged):
                if abs(start - merged_start) <= tolerance and abs(end - merged_end) <= tolerance:
                    # Merge the interval into the existing one
                    merged_interval = (min(start, merged_start), max(end, merged_end))
                    merged[i] = merged_interval
                    break
    
            if merged_interval is None:
                # No suitable merged interval found, create a new one
                merged.append((start, end))
    
        return merged


    def _merge_paths_within_interval(self, path_start_stop_dict):
        
        merged_dict = {}  # Create a new dictionary to store merged paths
        #print(f"{path_start_stop_dict=}")
        for pdb_path, (start, stop) in path_start_stop_dict.items():
            pdb_location, chain = pdb_path, os.path.basename(pdb_path)[5:-4]  #location + chain
            #print(f"{pdb_location=}, {chain=}")
            interval = (start, stop)
            
            if interval in merged_dict:
                merged_dict[interval].append((pdb_path, chain))
            else:
                merged_dict[interval] = [(pdb_path, chain)]
            
        return merged_dict


    def _flatten_nested_lists(self, lst):
        flattened = []
        for item in lst:
            if isinstance(item, list):
                flattened.extend(self._flatten_nested_lists(item))  #recursive call
            else:
                flattened.append(item)
        return flattened

    def _is_within_tolerance(self, interval1, interval2, tolerance):
    
        # Calculate the differences in starts and stops for both intervals
        start_diff = abs(interval1[0] - interval2[0])
        stop_diff = abs(interval1[1] - interval2[1])
        # Check if both differences are within the tolerance
        return start_diff <= tolerance and stop_diff <= tolerance


    def _save_human_readable(self, save=False):
        #helper function to save the result in human readable format.
        with open(os.path.join(self.log_dir, "domain_boundaries.txt"), "w") as db_out:
            for keys, vals in self.established_domain_dict.items():
                db_out.write(str(keys))
                db_out.write(":")
                db_out.write("\n")
                for pdb in vals:
                    pdb_name = os.path.basename(pdb[0])[:4]
                    pdb_chain = pdb[1]
                    db_out.write(pdb_name)
                    db_out.write("_")
                    db_out.write(pdb_chain)
                    db_out.write("\n")


    #{'5ltu_AB.pdb': [('A', 0.96), ('B', 0.96)], is oligodict

    def get_oligostates(self, num_most_common_oligostates=2):
        # First, we split into groups based on boundaries
        domains_grouped_dict = self.established_domain_dict
        # lets merge seq id and paths so we can handle them later
        domains_merge_dict = self._merge_dicts_range_seqid(domains_grouped_dict, self.oligodict)

        grouped_dict = self._split_and_group_by_chain(domains_merge_dict)

        # Initialize a dictionary to store the Counters for each range
        most_common_oligostates = {}

        # Initialize a dictionary to store the oligostate paths for each range
        oligostate_paths = {}

        for ranges, vals in grouped_dict.items():
            oligostate_counter = Counter()

            # Count occurrences for each oligostate
            for oligostate, entries in vals.items():
                oligostate_counter[oligostate] += len(entries)

            # Get the top oligostates up to the number specified by num_most_common_oligostates
            top_oligostates = oligostate_counter.most_common(num_most_common_oligostates)

            # Store the top oligostates and their counts
            most_common_oligostates[ranges] = top_oligostates

            # Store all paths associated with the top oligostates
            oligostate_paths[ranges] = [entry for oligostate, count in top_oligostates for entry in vals[oligostate]]

        self.most_common_filtered_oligostates = most_common_oligostates
        self.oligostates_filtered_paths = oligostate_paths

        # Display the major oligostates and their entries
        if self.logging:
            outp = os.path.join(self.log_dir, "filtered_oligostates.json")
            converted_dict =self._convert_keys_to_string(self.most_common_filtered_oligostates)
            self._logging(outp, dict_to_write=converted_dict)

        # Optionally, return the major oligostates and their entries if needed for further processing
        #return most_common_oligostates, oligostate_paths

        #lets fetch now for suitable templates for each of these states.
        self._process_templates()

    
    def _process_templates(self):
        #first we split into groups based on boundaries
        if self.oligostates_filtered_paths:
            domains_grouped_dict = self.oligostates_filtered_paths
        else:
            print("we have no self.oligostates_filtered_paths")
            return

        sorted_oligos_by_range = defaultdict()
        
        self.top_templates = defaultdict()
        
        for keys, vals in self.oligostates_filtered_paths.items():
            oligostate_dict = defaultdict(list)

            for (path, seqid) in vals:
                oligostate = len(os.path.basename(path)[5:-4])  # Extract oligostate
                oligostate_dict[oligostate].append((path, seqid))

            for oligostate in oligostate_dict:
                # Sort the list of tuples for each oligostate by seq_id
                oligostate_dict[oligostate] = sorted(oligostate_dict[oligostate], key=lambda x: x[1], reverse=True)
                
            sorted_oligostate_dict = OrderedDict(sorted(oligostate_dict.items()))

            formatted_key = "-".join([str(keys[0]), str(keys[1])])
            sorted_oligos_by_range[formatted_key] = sorted_oligostate_dict
            self.assemblied_structures = sorted_oligos_by_range

            return
            
            # Store the first hit of each oligostate for each range
            for oligostate, templates in sorted_oligostate_dict.items():
                # Initialize a dictionary for 'keys' if it does not exist
                if keys not in self.top_templates:
                    self.top_templates[keys] = {}
                    self.top_templates[keys][oligostate] = templates[0]  # Take the first template
                    
        if self.logging:
            outp = os.path.join(self.log_dir, "templates_oligos.json")
            converted_dict = self._convert_keys_to_string(self.templates_for_oligos)
            self._logging(outp, converted_dict)

    def _split_and_group_by_chain(self, input_dict):
        grouped_dict = {}
        for interval, pdb_list in input_dict.items():
            # Initialize a sub-dictionary for each interval
            grouped_dict[interval] = {}

            for pdb_path, seq_id in pdb_list:
                chain_length = len(os.path.basename(pdb_path)[5:-4])
                
                if chain_length not in grouped_dict[interval]:
                    grouped_dict[interval][chain_length] = []

                grouped_dict[interval][chain_length].append((pdb_path, seq_id))
        return grouped_dict

    
    def _logging(self, outp, dict_to_write):
        '''Logger to keep track of changes and get infos about potential debugging'''
        with open(outp, "w") as json_fh:
            json.dump(dict_to_write, json_fh, indent=4, default=str)
  
    def _convert_keys_to_string(self, dictionary):
        """Converts all the keys of the given dictionary to strings."""
        converted_dict = {}
        for key, value in dictionary.items():
            # Convert the key to a string, you might want to customize this formatting
            str_key = '_'.join(map(str, key)) if isinstance(key, tuple) else str(key)
            converted_dict[str_key] = value
        return converted_dict

In [5]:
templates = ['5ltu_A', '5ltu_B', '7nnj_A', '7nnj_B', '2duk_A', '2duk_B', '3mcf_A', '3mcf_B', '7tn4_A', '2q9p_A', '2fvv_A', '6pck_A', '6pcl_A', '6wo7_A', '6wo8_A', '6wo9_A', '6woa_A', '6wob_A', '6woc_A', '6wod_A', '6woe_A', '6wof_A', '6wog_A', '6woh_A', '6woi_A', '7aut_A', '7aui_A', '7auk_A', '7aul_A', '7aum_A', '7aun_A', '7auo_A', '7aup_A', '7auq_A', '7aur_A', '7aus_A', '7auu_A', '7auj_A', '3h95_A', '3i7u_A', '3i7u_B', '3i7u_C', '3i7u_D', '3i7v_A', '3i7v_B', '4hfq_A', '4hfq_B']
seq_id = [0.96, 0.96, 0.956, 0.956, 0.967, 0.967, 0.952, 0.952, 0.787, 0.787, 0.787, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.81, 0.4, 0.395, 0.395, 0.395, 0.395, 0.395, 0.395, 0.395, 0.395, 0.395, 0.395, 0.419, 0.388, 0.295, 0.438, 0.438, 0.438, 0.438, 0.438, 0.438, 0.355, 0.355]
query_start = [1, 1, 7, 7, 9, 9, 18, 18, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 29, 30, 30, 30, 30, 30, 30, 31, 31]
query_end = [181, 181, 147, 147, 147, 147, 145, 145, 157, 157, 157, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 149, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 85, 91, 128, 82, 82, 82, 82, 82, 82, 97, 97]
temp_start = [1, 1, 1, 1, 1, 1, 2, 2, 1, 23, 23, 1, 1, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 36, 13, 13, 13, 13, 13, 13, 78, 78]
temp_end = [180, 180, 140, 140, 138, 138, 129, 129, 156, 178, 178, 148, 148, 169, 169, 169, 169, 169, 169, 169, 169, 169, 169, 169, 169, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 108, 102, 108, 139, 63, 63, 63, 63, 63, 63, 148, 148]
work_dir = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline"
script_dir = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/scripts"
# in the near future: supply here with a uniprot id! and then fetch from there the conservation

#DownloadPipe
Downloader = DownloadPipe(templates=templates[0:9], work_dir=work_dir, seq_id=seq_id[0:9], script_dir=script_dir)
Downloader.paralellized_download()
Downloader.retrieve_meta() 
Downloader.setup_cutoff(cutoff=6, apply_filter=True)

struct = Downloader.filtered_structures

#PDB_builder block
PDB_Builder = PDBBuilder(work_dir=work_dir, structures=struct, remove_intermediates=True) #structures that are filtered
PDB_Builder.build_assembly()
PDB_Builder.create_domain_boundaries()
PDB_Builder.get_oligostates(num_most_common_oligostates=2)
assemblied_structures = PDB_Builder.assemblied_structures

This is overlap in the directory: {'2duk', '7nnj', '7tn4', '3mcf', '5ltu'}
we already have pdbs from the templates downloaded


In [6]:
assemblied_structures

defaultdict(None,
            {'1-151': OrderedDict([(1,
                           [('/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/2duk_A.pdb',
                             ('A', 0.967)),
                            ('/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/7nnj_A.pdb',
                             ('A', 0.956)),
                            ('/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/3mcf_A.pdb',
                             ('A', 0.952)),
                            ('/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/7tn4_A.pdb',
                             ('A', 0.787))]),
                          (2,
                           [('/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/5ltu_AB.pdb',
                             ('B', 0.96)),
                            ('/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/5ltu_AB.pdb',
                             ('A', 0

# CONTINUE HERE. 
Next step will be USAligner overwork and integration. 

In [94]:
USAligner = USAlign(work_dir=work_dir, script_dir=os.path.join(work_dir, "scripts"), structure_dict=assemblied_structures, 
                   cluster_min_identity=0.4,
                   num_strucs_per_cluster=10)

USAligner.USAlign_run()
USAligner.filter_results(tm_cutoff=0.7,rmsd_min_cutoff=0, rmsd_max_cutoff=10, log_file=True)
USAligner.setup_oligo_directories()
#USAligner.oligo_split_dictv
#result_dict = USAligner.result_dict
USAligner.multisequence_alignment(run_all_potential_oligos=True, log_file=True)

Log file saved as '/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/log_files/msa_seq_dict.json'.


In [ ]:
for keys, vals in USAligner.msa_seqs.items():
    print(keys)
    for pdb_code, seq in vals.items():
        print(pdb_code, seq)

In [93]:
class USAlign:
    '''Explain class USAlign'''
    def __init__(self, work_dir,
                 structure_dict,
                 script_dir=None,
                 top_templates=None,
                 cluster_min_identity=None, 
                 num_strucs_per_cluster=None,
                 logging=True):

        #current work dir
        self.work_dir = work_dir
        #location of all scripts required for USAlign
        self.script_dir = script_dir
        #template dict: key: range   value: dict : key: oligostate, value: tuple (list of pdbs, tuple(Chain, Seq_ID))
        self.structure_dict = structure_dict
        #this parameter sets a cutoff for structures to be included in the final ensemble.
        self.cluster_min_identity = cluster_min_identity
        #additional control to select how oligoclusters to be included for each range.
        self.num_strucs_per_cluster = num_strucs_per_cluster
        #
        self.logging = logging
        self.log_dir = os.path.join(work_dir, "log_files")
        #input normalized 
        self.oligo_split_dict = None
        self.normalized_dict = None
        self.filtered_cleaned_dict = None
        self.filtered_dict_after_usalign = None
        self.msa_seqs = None
        self.list_of_sizes_per_range_and_cluster_cleaned = None
        self.filtered_identity_clusters = None
        self.filtered_injector_dict = None
        self.list_of_sizes_per_cluster = None
        self.results_usalign = None
        self.result_dict = None
        self.multiseq_alignment = None
        self.pca_ready_ensemble = None
    
    def USAlign_run(self):
        '''Function call that runs USAlign.
        - First we convert our input into the correct input format.
        - If specified, we put constraints on the retrieved ensembles. 
        (minimum_template_identity:0.9, top_oligo_clusters=3)
        Means we only accept clusters that have at least a 0.9 or higher Seq identity in their cluster and top oligo_clusters=3 means
        the top 3 oligostates only.'''
        
        #this function converts the input to a dict with a value list.
        #self._convert_input_to_value_list()

        #If specified we further filter the subclusters that we find for each oligomeric state based on sequence identity.
        
        self.normalized_dict = self._restructure_input()

        if self.cluster_min_identity:
            #function to filter subclusters based on the passed threshold.
            self.filtered_identity_clusters = self._filter_subclusters()
        else:
            self.filtered_identity_clusters = self.normalized_dict

        #lets run USAlign
        self.filtered_cleaned_dict = self._prep_clusters_USAlign()

        #first dict is containing a dict: outer_key: range, inner key: oligostate val: path, tm_score, rmsd after alignment.
        #second dict templates is a dict: key: outer_key: range, inner key: oligostate, val: path
        self.results_usalign = self._parallelized_execution_USAlign()

    
    def _restructure_input(self):

        """Input needs to be normalized"""

        oligomer_mapping = {
            "1": "monomer", "2": "dimer", "3": "trimer", "4": "tetramer",
            "5": "pentamer", "6": "hexamer", "7": "heptamer", "8": "oktamer",
            "9": "nonamer", "10": "decamer", "11": "undecamer", "12": "dodecamer",
            "13": "tridecamer", "14": "tetradecamer", "15": "pentadecamer",
            "16": "hexadecamer", "17": "heptadecamer", "18": "oktadecamer",
            "19": "nonadecamer", "20": "eicosamer", "21": "eicosameundamer",
            "22": "eicosadodamer", "23": "eicosatrimer", "24": "eicosatetramer"
        }

        filtered_dict = defaultdict(lambda: defaultdict(list))

        for range_key, oligostate_dict in self.structure_dict.items():
            for oligostate_key, structures in oligostate_dict.items():
                # Convert oligostate_key to string and fetch oligomer name, defaulting to "X-mer" if not found
                oligomer_name = oligomer_mapping.get(str(oligostate_key), "X-mer")
                for path, (chain, seq_ID) in structures:
                    filtered_dict[range_key][oligomer_name].append((path, chain, seq_ID))

        filtered_dict = self._flatten_dict(filtered_dict)
        # this filtered_dict flattened structure has keys: start-stop-oligomer values: list of tuples (path, chain, seqid)
        return filtered_dict


    def _flatten_dict(self, nested_dict):
        flat_dict = {}
        for outer_key, inner_dict in nested_dict.items():
            for inner_key, value_list in inner_dict.items():
                combined_key = f"{outer_key}-{inner_key}"
                flat_dict[combined_key] = value_list

        return flat_dict

    
    def _parallelized_execution_USAlign(self):
        
        usalign_results = defaultdict(list)

        #defaultdict(<class 'list'>, {'1-151-monomer': [('/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/2duk_A.pdb', 'A', 0.967)
        with ProcessPoolExecutor() as executor:
            for range_oligo, path_chain_seqid_lst in self.filtered_cleaned_dict.items():
                range = range_oligo.split("-")[:2]
                oligo_state = range_oligo.split("-")[2]
                #print(f"{range=}, {oligo_state=}")
                if len(path_chain_seqid_lst) > 1:
                        template = path_chain_seqid_lst[0][0]
                        queries = [item[0] for item in path_chain_seqid_lst[1:]] 
                        # Initialize the futures list for this oligo_state
                        futures = []
                        if oligo_state == 1:  # Monomeric
                            for query in queries:
                                future = executor.submit(self._run_usalign_monomer, template, query)
                                futures.append(future)
                        else:  # Oligomeric
                            for query in queries:
                                future = executor.submit(self._run_usalign_oligomer, template, query)
                                futures.append(future)

                    
                        # Collecting results for each oligo_state
                        oligo_state_results = []
                        for future in as_completed(futures):
                            try:
                                result = future.result()
                                if result:
                                    oligo_state_results.append(result)
                            except Exception as e:
                                print(e)
                                continue
                                
                        # Store results in the dictionary
                        usalign_results[range_oligo] = (template, oligo_state_results)
                        #print(f"{range_oligo}: {usalign_results[range_oligo]=}")

        # Print or process range_results as needed
        return usalign_results

    
    def _filter_subclusters(self):
         
        #print(self.cleaned_input)
        filtered_dict = defaultdict(list)
        # {'1-151-monomer': [('/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/2duk_A.pdb', 'A', 0.967)
        for range_key_oligomer, path_chain_seqid_lst in self.normalized_dict.items():
            if path_chain_seqid_lst:
                for path, chain, seq_id in path_chain_seqid_lst: 
                        if seq_id > self.cluster_min_identity:
                            filtered_dict[range_key_oligomer].append((path, chain, seq_id))
                    
        # Removing any range keys that no longer have any oligostates after filtering
        filtered_dict = {k: v for k, v in filtered_dict.items() if v}
        return filtered_dict

    def _run_for_all_oligos(self):

        dir_path_pdb_codes = defaultdict(list)

        for dir_path, pdb_template_pdb_query_tuple in self.oligo_split_dict.items():
            if pdb_template_pdb_query_tuple:
                # Extracts all query PDB codes using list comprehension
                queries = [os.path.basename(f)[:-4] for f in pdb_template_pdb_query_tuple[1]]

                # Extract the PDB code from the template
                template_code = os.path.basename(pdb_template_pdb_query_tuple[0])[:-4]

                # Combine the template code with the query codes
                pdb_codes = queries + [template_code]  # Append the template code to the list of query codes

                # Store the combined list in the dictionary keyed by directory path
                dir_path_pdb_codes[dir_path] = pdb_codes

        # Print the resulting dictionary after the loop
        return dir_path_pdb_codes
    

    def multisequence_alignment(self, directory=None, run_all_potential_oligos=False, log_file=True):

        dir_path_pdb_codes = self._run_multiseq_alignment(directory=directory, run_all_potential_oligos=run_all_potential_oligos)

        #print(f"{dir_path_pdb_codes}")
        #lets check if not None.
        if dir_path_pdb_codes and run_all_potential_oligos:
            fasta_paths, fasta_cleared_paths = self._execute_msa_all_dirs(dir_path_pdb_codes)
        else:
            #implement for later use if someone wants only 1 directory.
            #self._execute_msa_single_dir(dir_path_pdb_codes)
            pass

        sequences = self._read_msa_output_converter(fasta_paths)

        #here we store all MSA for all oligos
        merged_dict_all_oligos_msa = defaultdict(dict)
        #this deals with monomeric cases
        self._merge_both_oligo_dicts(merged_dict_all_oligos_msa, sequences)
        #this deals with the oligomeric cases from mustang
        self._merge_both_oligo_dicts(merged_dict_all_oligos_msa, fasta_cleared_paths)

        #lets store it in msa_seqs
        self.msa_seqs = merged_dict_all_oligos_msa

        #now lets also logg the results.
        if log_file:
            self._log_results(merged_dict_all_oligos_msa)
        


    def _merge_both_oligo_dicts(self, dict1, dict2):
        for path, pdb_dict in dict2.items():
            if path in dict1:
                # Update the existing dictionary with new PDB ID and sequence pairs
                dict1[path].update(pdb_dict)
            else:
                # Add new directory path as a key with its PDB ID and sequence pairs
                dict1[path] = pdb_dict

    def _convert_defaultdict_to_dict(self, d):
        """Recursively convert a nested defaultdict to a standard dictionary."""
        if isinstance(d, defaultdict):
            # Convert defaultdict to dict and recurse for nested structures
            return {k: self._convert_defaultdict_to_dict(v) for k, v in d.items()}
        elif isinstance(d, dict):
            # Recurse for standard dict
            return {k: self._convert_defaultdict_to_dict(v) for k, v in d.items()}
        else:
            # Return value as is (end of recursion)
            return d 

    def _log_results(self, d, log_file_name="msa_seq_dict.json"):
        
        """Store the converted dictionary in a JSON log file."""
        converted_dict = self._convert_defaultdict_to_dict(d)
        
        # Ensure the log directory exists
        os.makedirs(self.log_dir, exist_ok=True)
        
        # Define the full path for the log file
        filename = os.path.join(self.log_dir, log_file_name)
        
        # Write the converted dictionary to the JSON file
        with open(filename, 'w') as f:
            json.dump(converted_dict, f, indent=4)
        print(f"Log file saved as '{filename}'.")

    
    def _execute_msa_all_dirs(self, dir_path_pdb_codes):

        fasta_path_dict = defaultdict()
        fasta_clear_dict = defaultdict()
        
        for dir_to_check, path_lst in dir_path_pdb_codes.items():
            
            pdb_list = os.path.join(dir_to_check, "pdb_list.txt")

            oligo_state = dir_to_check.split(os.sep)[-2]  # monomer, oligomer etc.
            #dir_to_check='/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/monomer/1-151'
            
            with open(pdb_list, "w") as pdb_fh:
                for pdbs in path_lst:
                    pdb_fh.write(pdbs)
                    pdb_fh.write("\n")

            if oligo_state == "monomer":
            
                bash_tm_and_rmsd_calc = f"{self.script_dir}/USalign -dir {dir_to_check}/ {pdb_list} -suffix .pdb -mm 4 -outfmt 1"
                bash_command = bash_tm_and_rmsd_calc.split()
                #print(bash_command)
                try:
                    fasta_path = os.path.join(dir_to_check, "multiseq_fasta_output.txt")
                    with open(fasta_path, "w") as fh_results:
                        result = run(bash_command, stdout=fh_results, stderr=PIPE, universal_newlines=True)
    
                    fasta_path_dict[dir_to_check] = fasta_path
                    
                except Exception as error:
                    print(error)
            else:
                "oligomer!"
                """
                HERE we should run mustang... 
                first delete all TER statements besides the last one. then run mustang."""

                #this function removes all TER statements between chains to make the room for mustang.
                self._prep_for_mustang(dir_to_check)
                
                all_files = [f for f in os.listdir(dir_to_check) if os.path.isfile(os.path.join(dir_to_check, f))]
                # Filter out files that end with '.pdb'
                pdbs = [f for f in all_files if f.endswith(".pdb") and not f.startswith("multiseq")]
                path_set_lst = list(set(pdbs))
                paths_abs = [os.path.join(dir_to_check, f) for f in path_set_lst]
                #print(f"{paths_abs=}")

                #works but careful. first argument supplied to -i is the reference where ALL structures will be superposed against This should ideally be the highest seq id
                #needs to be implemented in the future maybe to specifically start with highest seq id structure as 1rst argument passed to mustang. 
                fasta_path = os.path.join(dir_to_check, "multiseq_fasta_output")
                bash_mustang = f"{self.script_dir}/mustang -i {' '.join(paths_abs)} -F fasta -o {fasta_path}"
                bash_command = bash_mustang.split()
                #print(f"{bash_command=}")
                try:
                    result = run(bash_command, stdout=PIPE, stderr=PIPE, universal_newlines=True)
                    fpath_results = f"{fasta_path}.afasta"
                    #lets read it in
                    fasta_clear_dict[dir_to_check] = self._read_fasta_to_dict_mustang(fpath_results)
                
                except Exception as error:
                    print(error)

        return fasta_path_dict, fasta_clear_dict

    def _prep_for_mustang(self, dir_to_check):
        for filename in os.listdir(dir_to_check):
            if filename.endswith(".pdb"):  # Check if the file is a PDB file
                filepath = os.path.join(dir_to_check, filename)
                with open(filepath, 'r') as file:
                    lines = file.readlines()
                    
                # Remove all TER lines
                #this seems to work!
                new_lines = [line for line in lines if line.strip()[:3] != "TER"]
                # Add TER at the end
                new_lines.append("TER\n")

                # Write the modified lines to the same file to overwrite or to a new file
                with open(filepath, 'w') as file:  # Overwrite the same file
                    file.writelines(new_lines)
                # Optionally, you can write to a new file if you want to keep the original files unchanged

    
    def _read_fasta_to_dict_mustang(self, file_path):
        sequences = {}  # Initialize an empty dictionary to store sequences
        current_id = None  # Keep track of the current sequence ID

        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()  # Remove any leading/trailing whitespace
                if line.startswith('>'):  # Check if the line is an ID line
                    current_id = line[1:]  # Remove the '>' and set as the current ID
                    sequences[current_id] = ''  # Initialize an empty sequence for this ID
                else:
                    sequences[current_id] += line

        return sequences

    
    def _read_msa_output_converter(self, fasta_path_dict):
        
        sequence_dict = defaultdict()

        #print(f"{fasta_path_dict=}")

        for dir_to_fasta, fastas in fasta_path_dict.items():
            if os.path.exists(fastas):
                sequences = defaultdict()
                with open(fastas, "r") as fh_inp:
                    for line in fh_inp:
                        # Ignore lines that start with "#" or empty lines
                        if not line.startswith(">") and not line.startswith("#") and line.strip():
                            # The previous line contained the PDB ID, process it here
                            # Ensure there's a current PDB ID to process and sequence data exists
                            if current_pdb_id and line.strip():
                                sequences[current_pdb_id] = line.strip()
                        elif line.startswith(">"):
                            # Extract the PDB ID, remove ":A" part, and strip whitespace
                            current_pdb_id = line.split()[0][1:].split(':')[0].strip()
                        else:
                            # Reset current PDB ID for non-sequence lines or new entries
                            current_pdb_id = None
            
                #self.multiseq_alignment = sequences
                #print(f"{sequences=}")
                #self._write_fasta(sequences, outdir=directory)
                # Print the sequences dictionary
                #for pdb_id, sequence in sequences.items():
                #    print(f"{pdb_id}: {sequence}")
                sequence_dict[dir_to_fasta] = sequences
            
            else:
                print(f"File {fasta_path} does not exist.")
        
        return sequence_dict
    

    def _run_multiseq_alignment(self, directory=None, run_all_potential_oligos=False):
        """This function takes as input a pdb directory and will perform structure based superposition. 
        The results will be stored in a dict."""
        #grab all pdbs from the directory.
        if run_all_potential_oligos:
            if self.oligo_split_dict:
                dir_path_pdb_codes = self._run_for_all_oligos()
                return dir_path_pdb_codes # Ensure that the function exits after this operation
            else:
                print("Cannot run all potential oligos without oligo_split_dict!")
                return  # Exit the function if oligo_split_dict is not available

        # This part of the code will only be reached if run_all_potential_oligos is False
        if directory:
            self._run_for_single_directory(directory)  # Assuming oligomer is relevant here
        else:
            print("Specify a directory for MSA computation.")
        
    def prepare_for_PCA(self, directory, msa_alignment,cutting="strict"):
        #now we leverage the seq alignment from USAlign.
        '''
        Cutting based on a structure based alignment.
        Here we offer two options: strict cutting or intelligent cutting.
        
        - strict cutting:
        We directly take the ensemble, check for gaps in aligned positions 
        and REMOVE the atoms of a column IF there is a gap in 1 position.

        - intelligent cutting: 
        We first try to find structures in the ensemble that introduce many gaps.    
        We proceed by removal of those structures, rerunning the alignment and then continue with normal strict cutting.
        This can significantly improve the number of retained positions in the final PCA, but comes at the cost of dropping potentially
        relevant structures.
        '''

        # Directory to store results
        save_dir = os.path.join(directory, "trimmed_strucs")
        
        if cutting == "strict":
            #print("we move into strict cutting")
            residues_to_keep = self._hard_trimmer(msa_alignment)
            if not os.path.exists(save_dir):
                os.mkdir(save_dir)
            #this will be used to run PCA.
            self.pca_ready_ensemble = save_dir
            self._select_proper_residues_after_trim(residues_to_keep, save_dir)

        elif cutting == "intelligent":
            #these strucs need to be removed.
            strucs_to_remove = self._soft_trimmer(msa_alignment)
            #and then we rerun usalign.
            #we remove the strucs that are found to introduce many gaps.
            self._hard_trimmer(msa_alignment)

    def _soft_trimmer(self, result_fasta:str):
        '''Explain soft trimmer'''
        gap_counter = defaultdict()
        ali = pytrimal.Alignment.load(result_fasta)
        trimmer = pytrimal.AutomaticTrimmer("gappyout")
        trimmed = trimmer.trim(ali)
        for name, seq in zip(trimmed.names, trimmed.sequences):
            structure_name = name.decode().rjust(6)
            gap_count = seq.count("-")
            gap_counter[structure_name] = gap_count

        gap_values = list(gap_counter.values())
        mean_gap_count = statistics.mean(gap_values)
        median_gap_count = statistics.median(gap_values)

        high_gap_count_sequence_ids = [seq_id for seq_id, gap_count in gap_counter.items() if gap_count > mean_gap_count]
        print(high_gap_count_sequence_ids)
        return high_gap_count_sequence_ids
    

    def _hard_trimmer(self, result_fasta:str):
        '''Explain hard timmer'''
        keep_res_dict = defaultdict()
        ali = pytrimal.Alignment.load(result_fasta)
        #trimm all gaps unless this leaves 10% of the original alignment.
        trimmer = pytrimal.ManualTrimmer(gap_threshold=1) #changed to 80% now as test.
        trimmed = trimmer.trim(ali)
        for name, seq in zip(trimmed.names, trimmed.sequences):
            keep_res_dict[name.decode().rjust(6)] = seq

        return keep_res_dict

    def _select_proper_residues_after_trim(self, keep_residue_dict:dict, save_dir:dir):
        '''Helper function that takes the proper residues after trimming.'''
        for pdb, seq in keep_residue_dict.items():
            #seq in keep_residue_dict is extracted from hard trimmer.
            full_p = os.path.join(self.work_dir, pdb)
            result_seq = self._grab_sequence_from_struc(full_p)
            positions_to_keep = self._find_positions(result_seq[0], seq)
            #print(f"this is seq to keep: {seq}, this is full_seq from structure: {result_seq}")
            #print(f"this is our savepath for cutted struc: {full_p}")
            self._select_trimmed_positions(path_to_pdb=full_p, list_to_keep=positions_to_keep, save_dir=save_dir)

    def _select_trimmed_positions(self, path_to_pdb:str, list_to_keep:str, save_dir):
        '''Helper function that select only specific residues based on list_to_keep.'''
        class CAlphaTrimmedPositions(Select):
            def __init__(self, list_to_keep_shifts, *args):
                super().__init__(*args)
                self.list_to_keep_shifts = list_to_keep_shifts
            #overload accept_residue inherited from Select with this conditional return
            def accept_atom(self, atom):
                return 1 if atom.id == "CA" else 0
            #overloaded to only accept positive residue numbering.
            def accept_residue(self, residue):      
                return 1 if residue.id[1] in self.list_to_keep_shifts else 0    

        #lets make a new dir where we store the trimmed strucs.
        pdb_name = os.path.basename(path_to_pdb)
        new_location = os.path.join(save_dir, pdb_name) #here we save
        parser = PDBParser(QUIET=True)
        prot_name = f"default"
        structure = parser.get_structure(prot_name, path_to_pdb)
        shift = [x.get_id()[1] for x in structure.get_residues()][0] #first residue number = shift
        list_to_keep_shifted = [x+shift-1 for x in list_to_keep] #correct for shift. lets try with shift
        io = PDBIO()
        io.set_structure(structure)
        io.save(new_location, CAlphaTrimmedPositions(list_to_keep_shifts=list_to_keep_shifted))
    
    def _find_positions(self, full_sequence, partial_sequence):
        i = 0  # Index for full_sequence
        j = 0  # Index for partial_sequence
        found_positions = []
        while i < len(full_sequence) and j < len(partial_sequence):
            if partial_sequence[j] == '-':  # Skip gap positions in partial_sequence
                j += 1
            elif partial_sequence[j] == full_sequence[i]:  # Match found
                found_positions.append(i)
                i += 1
                j += 1
            else:
                i += 1  # Advance in full_sequence if no match
        found_pos = [x + 1 for x in found_positions]  # Adjust for 1-based indexing
        return found_pos

    def _grab_sequence_from_struc(self, pdb):
        '''Helper function that fetches sequence based on the structure'''
        lst =  [('VAL',"V"), ('ILE',"I"), ('LEU',"L"), ('GLU',"E"), ('GLN',"Q"),
                        ('ASP',"D"), ('ASN',"N"), ('HIS',"H"), ('TRP',"W"), ('PHE',"F"), ('TYR',"Y"), 
                        ('ARG',"R"), ('LYS',"K"), ('SER',"S"), ('THR',"T"), ('MET',"M"), ('ALA',"A"), 
                        ('GLY',"G"), ('PRO',"P"), ('CYS',"C")]
        
        canonical_aas = defaultdict(lambda: "X", lst)
        parser = PDBParser(QUIET=True)
        pdb_name = os.path.basename(pdb)
        prot_name = f"default"
        structure = parser.get_structure(prot_name, pdb)
        struc_seq = [canonical_aas[x.get_resname()] for x in structure.get_residues()]
        struc_seq = "".join(struc_seq)
        return (struc_seq, pdb_name)
        
    def _write_fasta(self, sequence_dict:dict, outdir=None):

        if outdir is None:
            outdir = self.work_dir
            
        outfile = os.path.join(outdir, "multiseq_fasta.fa")
        #Here we store our output as fastas for downstream analysis.
        seq_records = []
        for pdb_id, seq in sequence_dict.items():
            seq_record = SeqRecord(Seq.Seq(seq), id=pdb_id, description="")
            seq_records.append(seq_record)

        with open(outfile, "w") as output_handle:
            SeqIO.write(seq_records, output_handle, "fasta")
        
    
    def filter_results(self, tm_cutoff=0.5, rmsd_min_cutoff=0.1, rmsd_max_cutoff=20, log_file=True):
        # Check if results are available
        filtered_result_dict = defaultdict()
        
        if self.results_usalign:
            for range_oligos, usalign_outp in self.results_usalign.items():
                # The template is the first element
                template = usalign_outp[0]
                # The rest of the elements are lists of tuples (path, tm, rmsd)
                list_of_tuples = usalign_outp[1]  # Assuming the second element is the list of tuples
    
                filtered_results = []
                for path_tm_rmsd in list_of_tuples:
                    #print(f"Processing tuple: {path_tm_rmsd}")
                    # Unpack the tuple
                    path, tm, rmsd = path_tm_rmsd
    
                    # Convert tm and rmsd to floats and apply filters
                    try:
                        tm = float(tm)
                        rmsd = float(rmsd)
                        if tm > tm_cutoff and rmsd_min_cutoff <= rmsd <= rmsd_max_cutoff:
                            filtered_results.append((path, tm, rmsd))
                    except ValueError as e:
                        print(f"Error converting TM or RMSD to float: {e}")
                        continue  # Skip to the next tuple if there's an error
    
                # Update the result dict if there are any filtered results
                if filtered_results:
                    filtered_result_dict[range_oligos] = (template, filtered_results)
    
            # Optionally log the filtered results
            if log_file:
                self._logger_us_results(filtered_result_dict)
    
            # Update the instance variable with the filtered results
            self.filtered_dict_after_usalign = filtered_result_dict
    
        return filtered_result_dict



    #helper function that either passes filtered or unfiltered results downstream
    def setup_oligo_directories(self):
        #setup directories from here.
        if self.filtered_dict_after_usalign:
            self._organize_and_shuffle(self.filtered_dict_after_usalign)
            #self._separate_oligostates(self.filtered_dict_after_usalign)
        else:
            self._organize_and_shuffle(self.results_usalign)
            #self._separate_oligostates(self.results_usalign)
            
    
    def _logger_us_results(self, result_dict:dict):

        regular_dict = dict(result_dict) # convert to normal dict
        with open(os.path.join(self.log_dir, "filtered_usalign_log.json"), "w") as json_file:
            json.dump(regular_dict, json_file, indent=4)
        
    def _prep_clusters_USAlign(self):

        filtered_cleaned_dict = defaultdict(list)

        for range_oligostate, path_chain_seqid_lst in self.filtered_identity_clusters.items():
            # Remove duplicates and sort by seqid in descending order
            unique_tuples = set(path_chain_seqid_lst)
            sorted_tuples = sorted(unique_tuples, key=lambda x: x[2], reverse=True)

            # Retain only the top 'num_strucs_per_cluster' entries
            top_sorted_tuples = sorted_tuples[:self.num_strucs_per_cluster]

            # Store the size (number of structures) for the 'range_oligostate'
            filtered_cleaned_dict[range_oligostate] = top_sorted_tuples
    
        # Convert the default dicts to regular dicts for the final result
        return filtered_cleaned_dict

    
    #def _parallelized_execution_USAlign(self):
        
        #print(self.num_top_clusters_per_range)
        #self._get_top_clusters_per_range()
        # comment
        #self._extract_files_per_cluster()
        #now we want to send this dict into a function that does something
        #self.results = self._run_usalign_singles() 

    def _run_usalign_monomer(self, template, path_struc_1):
    
        bash_tm_and_rmsd_calc = f"{self.work_dir}/USalign {path_struc_1} {template} -outfmt 2 -mol prot"
        bash_command = bash_tm_and_rmsd_calc.split()
        print(bash_command)
        try:
            result = run(bash_command, stdout=PIPE, stderr=PIPE, universal_newlines=True)
            tm_2, rmsd = self._get_tm_scores_and_rmsd(result.stdout)
        
            # TM-score=0.5, it corresponds to a P-value of 5.5 × 10−7  taken from 
            #figure 3 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2913670/ 
        
            if float(tm_2) > 0.5:  # Adjust the threshold as needed
                return path_struc_1, tm_2, rmsd
            else:
                print(f"we remove {path_struc_1} because of tm: {tm_2} and rmsd: {rmsd}")
                #os.remove(path_struc_1)
                return path_struc_1, tm_2, rmsd
            
        except Exception as error:
            print(error)
            return None

    def _run_usalign_oligomer(self, template, path_struc_1):

        # -outfmt 2 : tsv outfile, 
        # -mol prot : only consider protein
        # -ter 0:align all chains from all models (recommended for aligning biological assemblies, i.e. biounits)
        # -mm 1: alignment of two multi-chain oligomeric structures
    
        bash_tm_and_rmsd_calc = f"{self.work_dir}/USalign {path_struc_1} {template} -outfmt 2 -mol prot -ter 0 -mm 1"
        bash_command = bash_tm_and_rmsd_calc.split()
        #print(bash_command)
        try:
            result = run(bash_command, stdout=PIPE, stderr=PIPE, universal_newlines=True)
            tm_2, rmsd = self._get_tm_scores_and_rmsd(result.stdout)
            if float(tm_2) > 0.5:  # Adjust the threshold as needed
                return path_struc_1, tm_2, rmsd
            else:
                print(f"we remove {path_struc_1} because of tm: {tm_2} and rmsd: {rmsd}")
                return path_struc_1, tm_2, rmsd
        except Exception as error:
            print(error)
            return None



    def _get_tm_scores_and_rmsd(self, results:str):
        """ helper function to retrieve tm scores and rmsd
        We are only interested in TM_2 and RMSD.
        IF RMSD is HIGH and TM_2 HIGH that means we have a conformer.
        IF RMSD is LOW and TM_2 HIGH that means we have the same structure in the same conformer
        IF RMSD is HIGH and TM_2 LOW that means the structures are not related."""
    
        """['#PDBchain1', 'PDBchain2', 'TM1', 'TM2', 'RMSD', 'ID1', 'ID2', 'IDali', 'L1', 'L2', 'Lali\n/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN/2duk_A.pdb:A', '/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN/5ltu_A.pdb:A', 
        '0.8833', '0.9565', '0.94', '0.920', '1.000', '1.000', '138', '127', '127\n']"""
    
        res_list = results.split("\t")
        #this one is from the mobile protein
        tm_1 = res_list[12]
        #this one belongs to the target protein (the one we superimpose the mobile protein onto)
        tm_2 = res_list[13]
        #rmsd used to judge cutoff for trashing structures.
        rmsd = res_list[14]
        #we return tm_2 and rmsd
        return ((tm_2, rmsd))
    

    def _extract_files_per_cluster(self):
        
        files_per_cluster = {}
        for range_key, clusters in self.list_of_sizes_per_cluster.items():
            # Check if the range key exists in the filtered_injector_dict
            if range_key in self.filtered_injector_dict:
                files_per_cluster[range_key] = {}
                for cluster_id in clusters.keys():
                    # Fetch the file paths for each cluster id
                    files_per_cluster[range_key][cluster_id] = self.filtered_injector_dict[range_key][cluster_id]

        self.filtered_injector_dict = files_per_cluster

    def _get_top_clusters_per_range(self):
    
        top_clusters = {}
        for range_key, clusters in self.list_of_sizes_per_cluster.items():
            # Sort clusters based on size, and in case of a tie, use larger key
            sorted_clusters = sorted(clusters.items(), key=lambda x: (-x[1], -x[0]))
            # Take the top n clusters, where n is specified by num_top_clusters_per_range
            top_n_clusters = dict(sorted_clusters[:self.num_top_clusters_per_range])
            top_clusters[range_key] = top_n_clusters

        self.list_of_sizes_per_cluster = top_clusters
        

    def _organize_and_shuffle(self, dict_to_process):
    
        dir_info_dict = defaultdict(tuple)

        for range_oligostate, (template_path, pdb_details) in dict_to_process.items():
            # Extract range and oligostate from the key
            range_key, oligostate = range_oligostate.rsplit('-', 1)
    
            # Define paths for oligostate and range directories
            oligo_dir = os.path.join(work_dir, oligostate)
            range_dir = os.path.join(oligo_dir, range_key)
    
            # Create the directories if they don't exist
            os.makedirs(range_dir, exist_ok=True)
    
            # Copy the template PDB into the range directory
            shutil.copy(template_path, range_dir)
            template_dst_path = os.path.join(range_dir, os.path.basename(template_path))
    
            # Initialize a list to store the paths of other structures
            other_structures = []
    
            # Copy other PDBs associated with the current range_oligostate into its directory
            for pdb_path, tm, rmsd in pdb_details:
                shutil.copy(pdb_path, range_dir)
                pdb_dst_path = os.path.join(range_dir, os.path.basename(pdb_path))
                other_structures.append(pdb_dst_path)
    
            # Store the template and list of other structures in the dictionary
            dir_info_dict[range_dir] = (template_dst_path, other_structures)
    
        # At this point, dir_info_dict contains the required information
        # and can be used as needed
        self.oligo_split_dict = dict(dir_info_dict)  # Convert defaultdict to dict if necessary


    # At this point, all required directories are created and PDB files are organized accordingly
    def _separate_oligostates(self, dict_to_process):

        #lets separate our found results and make new directories.   
        oligodirdict = defaultdict()
        
        oligos = defaultdict(dict)

        print(f"{dict_to_process=}")
        #for each range we go through all potential oligos.
        for ranges, dicts in dict_to_process.items():
            for key, value in dicts.items():  #dicts = dict with : key = oligostate, value = paths 
                oligos[ranges][key] = value
                    
        self.oligomers = oligos
        #now lets make the dirs.

        self._shuffle(oligos)


    def _shuffle(self, dict_to_shuffle):
        # Initialize the new dictionary structure for oligostates, ranges, and PDB paths
        new_dict = defaultdict(lambda: defaultdict(list))
        
        for range, oligo_data in dict_to_shuffle.items():
            for oligo, path_tm_rmsd in oligo_data.items():
                # Create the main directory for each "oligostate" if it doesn't exist
                oligo_path = os.path.join(work_dir, oligo)
                if not os.path.exists(oligo_path):
                    os.mkdir(oligo_path)
        
                # Create a subdirectory for each "range" within the "oligostate" directory
                range_path = os.path.join(oligo_path, range)
                if not os.path.exists(range_path):
                    os.mkdir(range_path)
            
                # Copy files associated with the current range into its directory
                for (path, tm, rsmd) in path_tm_rmsd:
                    shutil.copy(path, range_path)  # Copy file to the new location
                    
                    # Append only the PDB paths to the list associated with each range within each oligostate
                    new_dict[oligo][range].append((path, tm, rsmd))
        
        # Convert the defaultdict to a regular dict for the final output, if preferred
        final_dict = {oligo: dict(ranges) for oligo, ranges in new_dict.items()}
        
        # Now 'final_dict' contains the structure you want, and can be used as needed
        self.result_dict = final_dict
        #return final_dict

In [20]:
#dir_to_check = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/dimer/1-151"

#USAligner = USAlign(work_dir=work_dir, script_dir=os.path.join(work_dir, "scripts"), structure_dict=assemblied_structures, 
#                   cluster_min_identity=0.4,
#                   num_strucs_per_cluster=10)


#USAligner._prep_for_mustang(dir_to_check=dir_to_check)

In [210]:
class ModellerRepairEnsemble:
    '''
    Modeller Class that can be used to repair structures if needed.
    Returned structures should be compatible with format for downstream analysis.
    '''
    def __init__(self, work_dir, ensemble_dict):

        self.repair_ensemble = ensemble_dict
        self.work_dir = work_dir
        self.monomers = None
        self.oligomers = None
        self.monomeric_templates= None
        self.oligomeric_templates = None
        self.usalign_exe = os.path.join(work_dir, "USalign")
        self.repairable_structures = None
        self.repaired_monomers = None
        self.repaired_oligomers = None

    
    def _separate_oligostates(self):

        lst = [("1",'monomer'),("2", 'dimer'),("3", 'trimer'),("4", 'tetramer'),("5", 'pentamer'),
           ("6", 'hexamer'),("7", 'heptamer'),("8", 'oktamer'),("9", 'nonamer'),("10", 'decamer'),
           ("11", 'undecamer'),("12", 'dodecamer'),("13", 'tridecamer'),
           ("14", 'tetradecamer'),("15", 'pentadecamer'),("16", 'hexadecamer'),
           ("17", 'heptadecamer'),("18", 'oktadecamer'),("19", 'nonadecamer'),("20", 'eicosamer'),
           ("21", 'eicosameundamer'),("22", 'eicosadodamer'),("23", 'eicosatrimer'),("24", 'eicosatetramer')]

        oligodirdict = defaultdict(lambda: "X-mer", lst)

        
        oligos = defaultdict(dict)
        monos = defaultdict(dict)
        for ranges, dicts in self.repair_ensemble.items():
            for key, value in dicts.items():
                if key == 1:
                    monos[ranges][oligodirdict[str(key)]] = value
                else:
                    oligos[ranges][oligodirdict[str(key)]] = value

        self.monomers, self.oligomers = monos, oligos

    def _shuffle(self, dict_to_shuffle):


        lst = [("1",'monomer'),("2", 'dimer'),("3", 'trimer'),("4", 'tetramer'),("5", 'pentamer'),
           ("6", 'hexamer'),("7", 'heptamer'),("8", 'oktamer'),("9", 'nonamer'),("10", 'decamer'),
           ("11", 'undecamer'),("12", 'dodecamer'),("13", 'tridecamer'),
           ("14", 'tetradecamer'),("15", 'pentadecamer'),("16", 'hexadecamer'),
           ("17", 'heptadecamer'),("18", 'oktadecamer'),("19", 'nonadecamer'),("20", 'eicosamer'),
           ("21", 'eicosameundamer'),("22", 'eicosadodamer'),("23", 'eicosatrimer'),("24", 'eicosatetramer')]

        oligodirdict = defaultdict(lambda: "X-mer", lst)


        
        dirs_to_check = []
        
        for keys, vals in dict_to_shuffle.items():
            print(keys)
            keys = keys.split(",") #CONVERT tuple to a string.
            start, stop = keys[0][1:], keys[1][:-1]
            print(start)
            print(stop)
            new_range_obj = start+"-"+stop
            dir_path = os.path.join(work_dir, new_range_obj)
            if not os.path.exists(dir_path):
                #get rid of whitespace which makes issues later downstream
                os.mkdir(dir_path)
                
            for oligo, path_tm_rmsd in vals.items():
                print(oligo)
                oligo_path = os.path.join(work_dir, new_range_obj, oligodirdict[str(oligo)])
                if not os.path.exists(oligo_path):
                    os.mkdir(oligo_path)
                dirs_to_check.append(oligo_path) #this is a list that contains the locations of all directories that will be visited during repair.
                for path, _, _ in path_tm_rmsd:
                    shutil.copy(path, oligo_path) #new location
        
        return dirs_to_check

    
    def repair_structures(self):
        # first the monomeric case.
        self._separate_oligostates()
        
        # run now repairs on both.
        if self.monomers:
            #we need a separate environment for modeller etc.
            self.monomer_dirs = self._shuffle(self.monomers)
            oligo_dir_dict = defaultdict()
            if self.monomer_dirs:
                for directory in self.monomer_dirs:
                    #check which pdbs need repair.
                    pdb_to_check = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".pdb")] 
                    #first get the gap ranges
                    with ProcessPoolExecutor() as executor:
                        gap_list = list(executor.map(self._gap_localization_1, pdb_to_check))
                    #each gaplist member is a tuple of (path, list of gaps)
                    # keys: templates:   inner keys: the targets inner vals: their seqid and their TM score. which we will now filter and take the top X target to repair our structure.
                    
                    #print(f"{gap_list=}")
                    
                    suitable_templates = self._get_templates_for_monomeric_multitemplate_modelling(gap_list)

                    #print(f"{suitable_templates=}")
                    # now lets go through each and every of them an check if there are gaps.
                    
                    potential_template_dict = self._check_repairability(suitable_templates, gap_list)

                    #print(f"{potential_template_dict=}")
                    
                    #next step is repair. if empty we skip. else we go through all repairable structures.
                    if potential_template_dict:
                        with ProcessPoolExecutor() as executor:
                            key, value = zip(*potential_template_dict.items()) #unpack first 
                            try:
                                print("we try intelligent_monomeric_repair. ")
                                results = list(executor.map(self._intelligent_monomeric_repair, key, value))
                            except Exception as e:
                                results = []
                                print(e)
                        oligo_dir_dict[directory] = results
                        
                    else:
                        print(f"We have no repaired structures for directory: {directory}")
                        oligo_dir_dict[directory] = []
                        
            #this is a dict of paths for each oligostate and range as a instance of the class saved.
            self.repaired_monomers =  oligo_dir_dict


        
        
        
        if self.oligomers:
            #we need a separate environment for modeller etc.
            self.oligomer_dirs = self._shuffle(self.oligomers)

            oligo_dir_dict = defaultdict()
            if self.oligomer_dirs:
                for directory in self.oligomer_dirs:
                    #check which pdbs need repair.
                    pdb_to_check = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".pdb")] 


                    """Lets implement this later. First we just take all structures as they are and continue."""
                    oligo_dir_dict[directory] = pdb_to_check

                    
                    #first get the gap ranges
                    with ProcessPoolExecutor() as executor:
                        gap_list = list(executor.map(self._gap_localization_1_oligomeric, pdb_to_check))
        

                    suitable_templates = self._get_templates_for_oligo_multitemplate_modelling(template_path=template_path,
                                                                    potential_templates=template_backup_list,
                                                                    gap_dict=gap_dict)


            self.repaired_oligos = oligo_dir_dict


    def _check_repairability(self, template_dict: dict, gap_list: list, min_seq_id: float = 0.6) -> dict:
        """
        Checks each structure's eligibility for repair based on gap alignment and minimum sequence identity.
    
        Args:
        - template_dict (dict): A dictionary where keys are template paths and values are dicts containing template details.
        - gap_list (list): A list of tuples representing the paths and gaps in the structure to be repaired (path, [(start, stop), ...]).
        - min_seq_id (float): The minimum sequence identity required for a template to be considered suitable.
    
        Returns:
        - dict: A dictionary of potential templates that can be used for repair.
        """
        
        potential_template_dict = defaultdict(list)
        for (temp_path, temp_gaps) in gap_list:
            for template_path, details in template_dict.items():
                # Assume details structure is a dict with keys as some identifiers and values as tuples of details
                for key, val in details.items():
                    try:
                        # Attempt to unpack expecting 3 elements; adjust according to your data structure
                        tm_score, seq_id, template_gaps = val[0][0], val[0][1], val[1]
                    except ValueError:
                        # Handle cases where unpacking fails due to unexpected structure
                        print(f"Skipping {key} due to unpacking error: expected 3 values, got {len(val)}")
                        continue
                        
                    #print(seq_id, min_seq_id)
                    if float(seq_id) > min_seq_id and not self._has_large_gap_overlap(temp_gaps, template_gaps):
                        #print(seq_id, min_seq_id)
                        #print(temp_path, template_path)
                        potential_template_dict[(temp_path, temp_gaps)].append((template_path, (tm_score, seq_id, template_gaps)))
        
        return potential_template_dict

    def _has_large_gap_overlap(self, gaps1: list, gaps2: list) -> bool:
        """
        Determines if there is a large overlap in gaps between two structures.
    
        Args:
        - gaps1 (list): The gaps in the first structure as a list of tuples (start, stop).
        - gaps2 (list): The gaps in the second structure as a list of tuples (start, stop).
    
        Returns:
        - bool: True if there is a large overlap, False otherwise.
        """
        for start1, end1 in gaps1:
            for start2, end2 in gaps2:
                if start1 <= end2 and start2 <= end1:  # Check if gaps overlap
                    return True  # Overlapping gaps found
        return False  # No overlapping gaps




    
    def _gap_localization_1(self, pdb_path: str):
        """Helper function to compute the start and stops of gaps 
        for later potential reconstruction."""
    
        gap_ranges = []
        parser = PDBParser(QUIET=True)
        test_structure = parser.get_structure("test", pdb_path)
        
        test_res = test_structure.get_residues()
        start = end = None
    
        for res in test_res:
            res_id = int(res.get_id()[1])
            if end is None:
                start = end = res_id
            elif res_id == end + 1:
                end = res_id
            else:
                if start != end:
                    gap_ranges.append((start, end))
                start = end = res_id
    
        if start is not None and start != end:
            gap_ranges.append((start, end))

        if not gap_ranges:
            return (pdb_path, [])  # Return an empty list if there are no gaps

        # Convert the list of gap ranges to a list of gap tuples
        gap_tuples = [(start, end) for start, end in gap_ranges]
        merged_gaps = [(1, gap_tuples[0][0])] + [(gap_tuples[i][1], gap_tuples[i + 1][0]) for i in range(len(gap_tuples) - 1)]
        return (pdb_path, merged_gaps)


    

    def _get_templates_for_monomeric_multitemplate_modelling(self, pdb_gap_list):
        parser = PDBParser()
        suitable_templates = {}

        for template_index, (template_path, tmp_gap) in enumerate(pdb_gap_list):
            template_results = {}

            for target_index, (target_path, gaps) in enumerate(pdb_gap_list):
                if target_index != template_index:  # Avoid self-comparison
                    score = self._align(target_path, template_path)
                    template_results[target_path] = (score, gaps)

            suitable_templates[template_path] = template_results

        #print(suitable_templates)
        # outer keys: templates, inner keys: target_structures. inner values: (seq_id, tm., gaps) 
        return suitable_templates
    
    
    def _gap_localization_1_oligomeric(self,pdb_path: str):
        """Helper function to compute the start and stops of gaps for later potential reconstruction."""
        
        full_gaps_dict = defaultdict()
        gap_dict = defaultdict(list)
        parser = PDBParser(QUIET=True)
        test_structure = parser.get_structure("test", pdb_path)
        
        for model in test_structure:
            for chain in model:
                for residues in chain:
                    resnum = residues.get_id()
                    gap_dict[chain.id].append(resnum[1])
    
        #now we have all chains and all sequence nums from start to finish
        chain_to_query = []
    
        for chains, residues in gap_dict.items():
            chain_to_query.append(chains)
            
            start = end = None
            gap_ranges = []
            for res in residues:
                if end is None:
                    start = end = res
                elif res == end + 1:
                    end = res
                else:
                    if start != end:
                        gap_ranges.append((start, end))
                    start = end = res
    
            if start is not None and start != end:
                gap_ranges.append((start, end))
    
            gap_tuples = [(start, end) for start, end in gap_ranges]
            merged_gaps = [(1, gap_tuples[0][0])] + [(gap_tuples[i][1], gap_tuples[i + 1][0]) for i in range(len(gap_tuples) - 1)]
            full_gaps_dict[chains] = merged_gaps
        
        return full_gaps_dict, chain_to_query



    

    def _align(self, target_path, template_path):
        
        # Implement the logic to align the target_path with the template_path
        # focusing on the region defined by 'gap', and return a scoring metric
        bash_tm_and_rmsd_calc = f"{self.usalign_exe} {target_path} {template_path} -TMscore 0 -outfmt 1"
        bash_command = bash_tm_and_rmsd_calc.split()
        
        result_scores_for_templates = defaultdict()
        
        try:
            result = run(bash_command, stdout=PIPE, stderr=PIPE, universal_newlines=True)
    
            #print(result.stdout)
            result_list = self._get_aligned_fastas(result.stdout)  # gap list not used here... can be removed in future version.

            #print(result_list)
            return result_list
            
        except Exception as e:
            
            print(e)
            return {}
            


    def _get_aligned_fastas(self, result):

        lines = result.strip().split('\n')
        #this is gap_list : [(1, 3), (263, 276), (396, 407)]
        #print("this is gap_list :", gap_list)
        result_list = []
        #print(lines)
        # Initialize an empty dictionary
        template_header = lines[2].split("\t")
        #template_full_seq = lines[3]  #replace - to get the full seq.
        template_seq_id = float(template_header[3].split("=")[-1])
        template_tm_score = float(template_header[4].split("=")[-1])
        template_pdb_path = template_header[0].replace(">","")
        template_pdb_path = template_pdb_path.split(":")[0]
    
        potential_template_header = lines[0].split("\t")
        #potential_template_full_seq = lines[1]
        potential_template_seq_id = float(potential_template_header[3].split("=")[-1])
        potential_template_tm_score = float(potential_template_header[4].split("=")[-1])
        potential_template_pdb_path = potential_template_header[0].replace(">","")
        potential_template_pdb_path = potential_template_pdb_path.split(":")[0]
        #print(template_seq_id, template_tm_score)
        #print(potential_template_seq_id, potential_template_tm_score)
        #result_dict[potential_template_pdb_path] = (potential_template_seq_id, potential_template_tm_score, potential_template_full_seq)
        #result_dict[template_pdb_path] = (template_seq_id, template_tm_score, template_full_seq)
        return template_seq_id, template_tm_score
    
    def _score_meets_criteria(self, score):
        # Implement the logic to check if the score meets your criteria
        # for a suitable template
        pass


    


    def _intelligent_oligomeric_repair(self, path_to_pdb:str, gap_list:list, selected_templates:list,chains_query:list):
    
        """
        Function repairs structures with gaps less than 8 residues per gap.
    
        Args:
        - path_to_pdb (str): Path to the folder containing PDB files.
        - stop_pos (int): Stop position.
        - main_prot_seq (str): Main protein sequence.
        - use_main (bool): Whether to use the main protein.
    
        Output:
        Repaired structures.
        """
    
        #new_template_path='/home/micnag/test_modeller_oligomer/6hyr/6hyr.pdb'
        #,gap_list_all_chains=[[(1, 5), (96, 116)], [(1, 5)], [(1, 5)], [(1, 5)], [(1, 5)]], 
        #repair_templates_non_redundant={'/home/micnag/test_modeller_oligomer/6hz3_A.pdb'}







        
        print("inside oligomeric_repair")
        log.none()  # no stdout spam
        env = Environ()  # setup env for modelling
        aln = Alignment(env)  # setup the alignment
        mdl = Model(env)  # setup the model
    
        #path : /home/micnag/bioinformatics/.../monomer/pos/2duk_A.pdb
        # current working directory
        pdb_id_target = os.path.basename(path_to_pdb) #results in 2duk_A.pdb
    
        #pdb_id_target='6hyr.pdb'
        #print(f"{pdb_id_target=}")
        
        pdb_id_chain = chains_query #this corresponds to all chains in oligomeric struc. We shall also pass this
        #to our automodell derived child class.
    
        #we need first and last chain to set boundaries for our model 
        first_chain, last_chain = pdb_id_chain[0], pdb_id_chain[-1]
        
        #pdb_code_name is passed to modeller later.
        pdb_code_name = pdb_id_target[:-4] #2duk_ABCDEF
        
        pdb_4_digit_id = pdb_id_target[0:4] #2duk
    
        #print(f"{pdb_4_digit_id=}")
        
        #get uniprot_id second. HERE CHECK HOW WE GET X UNIPROT IDS FOR X PROTEINS IN A HETERO_OLIGOMER
        uniprot_id = return_uniprot_id_from_rcsb(pdb_4_digit_id)
    
        #print(f"{uniprot_id=}")
        #get associated fasta from uniprot id.
        fasta_seq = get_gene_fasta(uniprot_id)
    
        #print(f"{fasta_seq=}")
    
        #first_chain='A', last_chain='E', pdb_4_digit_id='6hyr', pdb_code_name='6hyr',uniprot_id='Q7NDN8', pdb_id_chain=['A', 'B', 'C', 'D', 'E']
        #print(f"{first_chain=}, {last_chain=}, {pdb_4_digit_id=}, {pdb_code_name=},{uniprot_id=}, {pdb_id_chain=}, {fasta_seq=}")
        """HERE WE NEED TO CHECK HOW TO MERGE OUR FASTAS.. ESPECIALLY CRUCIAL IF WE HAVE A MIXED OLIGOMER.
        SUBSEQUENT CHAINS NEEDS TO BE SEPARATED BY / """
        
        #if mixed oligomer this needs to be taken into account. 
        """WE NEED TO CHECK THIS IN THE FUTURE... ADAPT GET_GENE_FASTA for multiple seqs in hetero X mers."""
        
        merged_fasta = '/'.join([fasta_seq] * len(pdb_id_chain))
        
        #first we check what is the first ID in our struc. because otherwise we can miss structures that simply miss 
        #N terminus e.g gap 1-21 but they in fact start with residue 21 and are otherwise good or might just have small
        # gaps to fix otherwhere in the structure.
    
        start_stop_dict = defaultdict()
    
        temp_codes = []
        temp_abs_paths = []
        temp_chains = []
    
    
    
        #print(f"{merged_fasta=}")
    
        for structure in set(selected_templates):  #no duplicates here.
            #print(f"Structure: {structure} selected as template")
            
            temp_codes.append(structure.split("/")[-1][:-4]) #extract codes for all strucs. this works for oligomers.
            
            temp_abs_paths.append(structure) #grab abspath
            
            temp_chains.append(structure.split("/")[-1][5:-4]) #this are the chains we need.
    
    
        #print(f"{temp_codes=}, {temp_abs_paths=}, {temp_chains}")
    
        for temp_code, temp_paths, chains in zip(temp_codes, temp_abs_paths, temp_chains):
            if len(temp_code) <= 6:
                #append all start stop and shifts here.
                start_struc_temp, stop_struc_temp = get_struc_stop_oligomer(temp_paths)  #this works also for oligomers in theory. But it is questionable.. if chain A is 100 res and chain B is 40.
                #how should we give this info to modeller... 1:A until B:40 problably. include 
                
                start_stop_dict[temp_code] = ((start_struc_temp, stop_struc_temp, chains))
    
        start_struc_query, stop_struc_query = get_struc_stop_oligomer(path_to_pdb) 
        
        start_stop_dict[pdb_code_name] = ((start_struc_query, stop_struc_query, pdb_id_chain))
    
    
        #print(f"{start_stop_dict=}")
        
        #now the dict contains all paths and start stops.
        
        #this is the only shift we care about!
    
        #careful here.. shifts need to be taken into account FOR EACH CHAIN!
        
        shift = abs(1-start_struc_query)  #e.g 1- 8 abs means 7 shift.
        
        #if we find that the first end of gap corresponds to the start resi number of the pdb... we skip this gap.
    
        #would be better to check if one of the templates might cover this gap. But currently not implemented.
    
        #print(f"{shift=}")
    
    
        #gap_list=[((96, 116), 'A')]
        #print(f"{gap_list=}")
    
        """
    
        gap_list_all_chains=[[(1, 5), (96, 116)], [(1, 5)], [(1, 5)], [(1, 5)], [(1, 5)]]
        we enter oligomeric repair with: new_template_path='/home/micnag/test_modeller_oligomer/6hyr/6hyr.pdb',repairable_gaps=[((96, 116), 'A')], repair_templates_non_redundant={'/home/micnag/test_modeller_oligomer/6hz3_A.pdb'}
        inside oligomeric_repair
        Structure: /home/micnag/test_modeller_oligomer/6hz3_A.pdb selected as template
        temp_codes=['6hz3_A'], temp_abs_paths=['/home/micnag/test_modeller_oligomer/6hz3_A.pdb'], ['A']
        start_stop_dict=defaultdict(None, {'6hz3_A': (5, 315, 'A'), '6hyr': (5, 315, ['A', 'B', 'C', 'D', 'E'])})
        shift=4
        """
        
    
        
        """CONTINUE HERE AFTER COURSE TO PROCEED WITH HETEROMERIC OLIGOMER REPAIRS.  NOV 17 2023  18:33 """
        
    
        for gaps, chains in gap_list:
            #print(gaps, chains)
            
            found_N_terminus = False
            found_C_terminus = False
    
            #can only concatenate str (not "int") to str#
    
            #    n_terminus = [x for x in range(gap_list[0][0], gap_list[0][1]+1)] #including the last residue.
            #    c_terminus = [x for x in range(gap_list[-1][0], gap_list[-1][1]+1)] #including the last residue.
        
            #(96, 116) A
    
            if len(gaps) > 2: #means we have more than 1 gap. each gap = len 2
                n_terminus = [x for x in range(gaps[0][0], gaps[0][1]+1)] #including the last residue.
                c_terminus = [x for x in range(gaps[-1][0], gaps[-1][1]+1)] #including the last residue.
            else:
                n_terminus = None
                c_terminus = None
                #we dont deal with n or c terminus here if we only have 1 gap.
    
            #print(f"this is {n_terminus=}")
            #print(f"this is {c_terminus=}")
    
            max_n_terminus_found = None
            max_c_terminus_found = None
    
            #print(f"works until here inside intelligent oligomeric repair!!")
    
    
    
            for paths, (start, stop, chain) in start_stop_dict.items():
                # paths='6hz3_A',start=5, stop=315, chain='A'
                # paths='6hyr',start=5, stop=315, chain=['A', 'B', 'C', 'D', 'E']
                
                #print(f"{paths=},{start=}, {stop=}, {chain=}")
                #lets check for the n-terminus if we find something that can be repaired.
                
                if n_terminus:
                    if start < n_terminus[-1]:
                    
                        #then we set n terminus found because there is something to repair
                        found_N_terminus = True
                    
                        #but if we have no template... we cant repair beyond what we have and we stay with start.
                        if max_n_terminus_found == None or start < max_n_terminus_found:
                            max_n_terminus_found = start
                            
                #lets check for the c terminus in the same fashion
                if c_terminus: 
                    if stop > c_terminus[0]:
                    
                        #that means we have something that goes beyond the max available struc len.
                        found_C_terminus = True
                    
                        #but if we have no template, we still cant repair beyond.
                        if max_c_terminus_found == None or stop > max_c_terminus_found:
                            max_c_terminus_found = stop
                    
    
    
            #now lets check outside the loop
    
            if found_N_terminus == False and len(gaps) != 2: #means its not our only hit.
    
                if len(gaps) > 2:
    
                    gap_list = gaps[1:] #skip first tuple because we cant repair N terminus
                    
                else:
                    
                    print(f"We could not repair the only gap because we have no template. this is {gap_list=}")         
                    return
            
            if found_C_terminus == False and len(gaps) != 2:
                #this means we have no suitable template and we skip it.
                
                if len(gap_list) > 2:
                    
                    gap_list = gap_list[:-1]
                    
                else:
                    
                    print(f"We could not repair the only gap because we have no template. this is {gap_list=}")
                    return
    
            
            #lets grab the max range of gaps in our structure.
            if len(gaps) != 2:
                max_gap_range = sorted([y - x for x, y in gaps], reverse=True)
            else:
                max_gap_range = [gaps[1] - gaps[0]] #simple 1 gap, list for downstream compatibility
    
    
            if len(selected_templates) == 0 and max_gap_range[0] > 8:  
                #this last part checks for the case we have
                #no templates found but only small gaps of lenght < 8 and still want to repair.
                print("no suitable templates found.")
                return
    
        #print("we went through this chaos.")
    
        """Modeller aln.salign treats the FIRST structure provided as QUERY. so we need to load this one FIRST."""
        #code to be passed to mdl.read
    
        #start struc query is already above computed at the beginning so we re use it again here.
        #CONTINUE HERE
    
        """model_segment specifies the range we look into. Ideally we look for start - stop based on majority vote. Chain is always the same in monomeric"""
        mdl.read(file=pdb_code_name, model_segment=(f"{start_struc_query}:{pdb_id_chain[0]}", f"{stop_struc_query}:{pdb_id_chain[-1]}"))
    
        #pdb_code_name='6hyr',start_struc_query=5,pdb_id_chain[0]='A',stop_struc_query=315,pdb_id_chain[-1]='E'
        #print(f"{pdb_code_name=},{start_struc_query=},{pdb_id_chain[0]=},{stop_struc_query=},{pdb_id_chain[-1]=}")
        #append model object to alignment object.
    
        aln.append_model(mdl, align_codes=pdb_code_name, atom_files=pdb_code_name)
    
         
        for codes, (start_temp, stop_temp, chains) in start_stop_dict.items():
            #dont add our query again to the stack.
            if codes == pdb_code_name:
                continue
    
            #rest of templates...add to the stack.
            mdl.read(file=codes, model_segment=(f"{start_temp}:{chains[0]}", f"{stop_temp}:{chains[-1]}"))
            
            aln.append_model(mdl, align_codes=codes, atom_files=codes)
    
    
        #now add fasta sequence as last entry to the align model.
        with open(f"./{pdb_code_name}x.fasta", "w") as fastaout:
            fastaout.write(f">{pdb_code_name}x\n")
            fastaout.write(merged_fasta)
    
        aln_code = f"{pdb_code_name}x"
    
        #align fasta file to our alignment object which contains now a fasta sequence and a structure object.
        aln.append(file=f"./{pdb_code_name}x.fasta", align_codes=aln_code, alignment_format="fasta")
    
    
    
    
        #Overwrite MyModel and Inherit from AutoModel.
        class MyModel(AutoModel):
            def __init__(self, env, alnfile, knowns, sequence, gaps, chains, shift, **kwargs):
                super(MyModel, self).__init__(env, alnfile, knowns, sequence, **kwargs)
                self.gaps = gaps
                self.chain = chains
                self.shift = shift #shift because modeller always renumbers all stuff to be 1 based.
                
            def select_atoms(self):
                selections = []
                chain = self.chain
                #this needs to be adjusted for multi chain modells.
                #this is self.gaps: [((96, 116), 'A')]
                #print(f"this is self.gaps: {self.gaps}")
                for (start, end), chain in self.gaps:
                    if start > 1:
                        
                        #print(f"Selecting atoms for range {start-self.shift}:{chain} to {end-self.shift}:{chain}")
                        selection = self.residue_range(f'{start-self.shift}:{chain}', f'{end-self.shift}:{chain}')
                        selections.append(selection)
                        
                    else:
                        #print(f"Selecting atoms for range {start}:{chain} to {end-self.shift}:{chain}")
                        selection = self.residue_range(f'{start}:{chain}', f'{end-self.shift}:{chain}')
                        selections.append(selection)
    
                selected_atoms = Selection(*selections)
                
                # Combine all selections into a single Selection object
                print(f"Selected {len(selected_atoms)} atoms for optimization")
                
                return selected_atoms
    
    
        #setup environment dir for MODELLER. ACCEPTED current dir and previous dir.
        env.io.atom_files_directory = ['.','../.']
        
        #aln.malign3d(fit=True)
    
    
        # Additional debugging: Print alignment content
        #print("Alignment content before salign:")
        #for record in aln:
            #print(record.code)
    
        #align sequence to structure.
    
        #overhang = 0 because we are confident the structures as templates are suitable candidates (seq id > 0.8)
        # gap penalties are default.
        # alignment_type = progressive : each template pairwise against query comparison.
        # 
    
        
        aln.salign(overhang=0, gap_penalties_1d=(-450, -50), alignment_type="progressive", output="ALIGNMENT")
    
        #write out alignmentfile for automodell usage later
        aln.write(file=f"{pdb_code_name}.ali")
        
        #add template codes to the code list.
        alternate_templates = [x for x in temp_codes]
    
        # merge with our codes.
        full_knowns = (pdb_code_name, *alternate_templates) #empty list from alternate_templates
        
        #print(f"{full_knowns=}")
        #custom class
    
        # selected atoms do not feel the neighborhood
        #env.edat.nonbonded_sel_atoms = 2
    
        #here we need to rechain again and make sure that this is what modeller sees.. e.g BCD Will need to be ABC
    
        pdb_original = None #this is considered false by default == 1 == False
    
        letters = [chr(ord('A') + i) for i in range(26)]
        
        
        if pdb_id_chain != letters[0:len(pdb_id_chain)]:  # ABC for len 3 e.g
            #print(f"we are inside pdb_id_chain not fitting with modeller. : {pdb_id_chain=}")
            pdb_original = list(pdb_id_chain)
            #we set it to A for repair.. but afterwards we swap it back!
            pdb_id_chain = letters[0:len(pdb_id_chain)]
    
        #print(f"{pdb_id_chain=}")
    
        #print("works until here.")
    
        a = MyModel(env, alnfile=f"{pdb_code_name}.ali", knowns=full_knowns, sequence=aln_code,
                    gaps=gap_list, shift=shift, chains=pdb_id_chain)
    
    
        #print(f"ALIGN_CODES(1) = {a.alignment_codes[0]}")
        #a = AutoModel(env, alnfile=f"{pdb_code_name}.ali", knowns=full_knowns, sequence=aln_code)
    
        a.starting_model = 1
        a.ending_model = 1
    
        # Thorough MD optimization:
        #a.md_level = refine.slow
    
        # Repeat the whole cycle 2 times and do not stop unless obj.func. > 1E6
        #a.repeat_optimization = 2
    
        #env.libs.topology.make(aln)
        #env.libs.parameters.read(file='$(LIB)/par.lib')
        #mdl.generate_topology(aln[pdb_code_name])
        #a.generate_topology(aln[pdb_code_name])
        # Assign the average of the equivalent template coordinates to MODEL:
        #a.transfer_xyz(aln) #lets try this.
        
        # Get the remaining undefined coordinates from internal coordinates:
        #a.build(initialize_xyz=True, build_method='INTERNAL_COORDINATES')
    
        try:
            # Build the model(s)
            a.make();
            
        except Exception as e:
            print(f"An error occurred during modeling: {e}")
    
        #this worked.. now we need to clean all files that are no longer required
    
    
        try:
            #remove artifacts.
            #remove_repair_artefacts(pdb_basep=path_to_pdb, pdb_code_name=pdb_code_name)
    
            #ok now we need to check what the updated start stop range is!
    
            if max_n_terminus_found and max_n_terminus_found < start_struc_query:
                
                keep_start = max_n_terminus_found
                if not max_c_terminus_found:
                    keep_stop = stop_struc_query
                
                    
            if max_c_terminus_found and max_c_terminus_found > stop_struc_query:
                keep_stop = max_n_terminus_found
    
                if not max_n_terminus_found:
                    keep_start = start_struc_query
                    
            if not max_n_terminus_found and not max_c_terminus_found:
                keep_start, keep_stop = start_struc_query, stop_struc_query
            
    
            #here we can switch chain back to original if required:
    
            
            #print(f"{keep_start=}, {keep_stop=}")
            
            #this should select only from the start to end and excludes potentially repaired N and C termini that would only introduce more noise.
            select_c_alpha_and_correct_range(path_to_pdb, start=keep_start, stop=keep_stop)
    
            
            
            #not required!
            #renumber_structure_monomeric(path_to_pdb, start=start_struc, chain=pdb_id_chain)
            
        except Exception as e:
            print(e)

        return



    


    def _intelligent_monomeric_repair(self, repair_struc_gap_lst, temp_paths_tm_score_seq_id_template_gaps):
    
        """
        Function repairs structures with gaps less than 7 residues per gap.
    
        Args:
        - template_dict: key = (pdb to be repaired, temp_gaps), vals = (path_for_template_structure,  (tm_score, seq_id, template_gaps)) 

        We need to make sure that the selection of residues used for repair does not fall into the region of template_gaps.

        
        Attention:
    
        Modeller INTERNALLY RECHAINS AND RENUMBERS EVERY QUERY.
        ALL SINGLE CHAINS == A chain. irrespective of natural chain.. e.g if you supply chain B single chain it will go back to rechaining it to chain A.
        Also dangerous... it will renumber structures from 1... so you need to correct for this shift based on your desired selection target. 
        If you structure starts with residue 30... and you want to remodell a gap between 40-50 you need to take this into account. shift = abs(1-start_struc_query)
        
        Output:
        Repaired structures.
        """
        
        print(repair_struc_gap_lst)
        print(temp_paths_tm_score_seq_id_template_gaps)

        #lets unpack everything properly.

        pdb_id_target_path = repair_struc_gap_lst[0] #the path
        pdb_id_target_gaps = repair_struc_gap_lst[1] #the list of gaps to be checked

        template_paths = temp_paths_tm_score_seq_id_template_gaps[0] #the path to the templates
        template_gaps = temp_paths_tm_score_seq_id_template_gaps[1][2] # the list of gaps in the templates


        if len(pdb_id_target_gaps) == 0:
            #means we have nothing to add.
            return pdb_id_target_path 
        
        log.none()  # no stdout spam
        env = Environ()  # setup env for modelling
        aln = Alignment(env)  # setup the alignment
        mdl = Model(env)  # setup the model
        #path : /home/micnag/bioinformatics/.../monomer/pos/2duk_A.pdb
        # current working directory
        #print("works until here step 1:")
        pdb_id_target = os.path.basename(pdb_id_target_path) #results in 2duk_A.pdb
        pdb_id_chain = pdb_id_target[-5] #this corresponds to chain in single monomeric struc. We shall also pass this
        #to our automodell derived child class.
        #pdb_code_name is passed to modeller later.
        pdb_code_name = pdb_id_target[:6] #2duk_A
        pdb_4_digit_id = pdb_id_target[0:4]
        #print(f"this is {pdb_4_digit_id=}, {pdb_id_chain=}, {pdb_code_name=}")
        #print("works until here step 2:")
        #get uniprot_id second.
        uniprot_id = self._return_uniprot_id_from_rcsb(pdb_4_digit_id)
        #get associated fasta from uniprot id.
        fasta_seq = self._get_gene_fasta(uniprot_id)
    
        #first we check what is the first ID in our struc. because otherwise we can miss structures that simply miss 
        #N terminus e.g gap 1-21 but they in fact start with residue 21 and are otherwise good or might just have small
        # gaps to fix otherwhere in the structure.
    

        start_stop_dict = self._setup_and_prep_templates(template_paths)
        start_struc_query, stop_struc_query = self._get_struc_stop(pdb_id_target_path) 

        #leave this as this adds the QUERY to the start_stop_dic
        start_stop_dict[pdb_code_name] = ((start_struc_query, stop_struc_query, pdb_id_chain))

        
        #now the dict contains all paths and start stops.
        #this is the only shift we care about!
        shift = abs(1-start_struc_query)  #e.g 1- 8 abs means 7 shift.
        
        #if we find that the first end of gap corresponds to the start resi number of the pdb... we skip this gap.
        #would be better to check if one of the templates might cover this gap. But currently not implemented.
        #[(275, 284), (882, 889)]
        #print(f"this is {gap_list=}")
        
        found_N_terminus, found_C_terminus, pdb_id_target_gaps = self._check_terminus_coverage(pdb_id_target_gaps, start_stop_dict)
            
        #print(f"gaplist after cutting: {gap_list=}")
        
        #lets grab the max range of gaps in our structure.
        #max_gap_range = sorted([y - x for x, y in pdb_id_target_gaps], reverse=True)
    
        #check if we found suitable templates... IF NO and there are still only small gaps < 8 then we use the structure itself as template and still repair.
        #if len(selected_templates) == 0 and max_gap_range[0] > 8:  
            #this last part checks for the case we have
            #no templates found but only small gaps of lenght < 8 and still want to repair.
            #print("no suitable templates found.")
        #    return
            
        #elif len(selected_templates) == 0 and max_gap_range[0] < 8:
            #nothing to do since we always append our own structure to repair ensemble. But in this case we can continue.
            #print("we found gaps but they are small so we shall continue")
        #    pass
        
    
        """Modeller aln.salign treats the FIRST structure provided as QUERY. so we need to load this one FIRST."""
        #code to be passed to mdl.read
    
        #start struc query is already above computed at the beginning so we re use it again here.
        #CONTINUE HERE
    
        """model_segment specifies the range we look into. Ideally we look for start - stop based on majority vote. Chain is always the same in monomeric"""
        mdl.read(file=pdb_code_name, model_segment=(f"{start_struc_query}:{pdb_id_chain}", f"{stop_struc_query}:{pdb_id_chain}"))
        #append model object to alignment object.
        aln.append_model(mdl, align_codes=pdb_code_name, atom_files=pdb_code_name)
    
        #now lets add all the templates.
        #now we need to append all template strucs.
        
        for codes, (start_temp, stop_temp, chains) in start_stop_dict.items():
            #dont add our query again to the stack.
            if codes == pdb_code_name:
                continue
    
            #rest of templates...add to the stack.
            #print(f"{start_temp}:{chains}", f"{stop_temp}:{chains}")
            mdl.read(file=codes, model_segment=(f"{start_temp}:{chains}", f"{stop_temp}:{chains}"))  
            aln.append_model(mdl, align_codes=codes, atom_files=codes)
    
        #now add fasta sequence as last entry to the align model.
        with open(f"./{pdb_code_name}x.fasta", "w") as fastaout:
            fastaout.write(f">{pdb_code_name}x\n")
            fastaout.write(fasta_seq)
    
        aln_code = f"{pdb_code_name}x"
        #align fasta file to our alignment object which contains now a fasta sequence and a structure object.
        aln.append(file=f"./{pdb_code_name}x.fasta", align_codes=aln_code, alignment_format="fasta")
        
        class MyModel(AutoModel):
            def __init__(self, env, alnfile, knowns, sequence, gaps, chain, shift, **kwargs):
                super(MyModel, self).__init__(env, alnfile, knowns, sequence, **kwargs)
                self.gaps = gaps
                self.chain = chain
                self.shift = shift #shift because modeller always renumbers all stuff to be 1 based.
                
            def select_atoms(self):
                selections = []
                chain = self.chain
    
                #print(f"this is self.gaps: {self.gaps}")
                for start, end in self.gaps:
                    if start > 1:  #negative indx make problems so we start from 1 in this cases.
                        #print(f"Selecting atoms for range {start-self.shift}:{chain} to {end-self.shift}:{chain}")
                        selection = self.residue_range(f'{start-self.shift}:{chain}', f'{end-self.shift}:{chain}')
                        selections.append(selection)
                    else:
                        #print(f"Selecting atoms for range {start}:{chain} to {end-self.shift}:{chain}")
                        selection = self.residue_range(f'{start}:{chain}', f'{end-self.shift}:{chain}')
                        selections.append(selection)
                
                selected_atoms = Selection(*selections)
                # Combine all selections into a single Selection object
                #print(f"Selected {len(selected_atoms)} atoms for optimization")
                return selected_atoms
    
    
        #setup environment dir for MODELLER. ACCEPTED current dir and previous dir.
        env.io.atom_files_directory = [f'{self.work_dir}', f'../.']

        #aln.malign3d(fit=True)
        # Additional debugging: Print alignment content
        #print("Alignment content before salign:")
        #for record in aln:
            #print(record.code)
        #align sequence to structure.
        #overhang = 0 because we are confident the structures as templates are suitable candidates (seq id > 0.8)
        # gap penalties are default.
        # alignment_type = progressive : each template pairwise against query comparison.
        #    
        aln.salign(overhang=0, gap_penalties_1d=(-450, -50), alignment_type="progressive", output="ALIGNMENT")
        #write out alignmentfile for automodell usage later
        aln.write(file=f"{pdb_code_name}.ali")
        #add template codes to the code list.
        alternate_templates = [x for x in temp_codes]
        # merge with our codes.
        full_knowns = (pdb_code_name, *alternate_templates) #empty list from alternate_templates
        #print(full_knowns)
        #custom class
        # selected atoms do not feel the neighborhood
        #env.edat.nonbonded_sel_atoms = 2
        #check chain if not A... take A and afterwards change it back to original chain. 
        pdb_original = None #this is considered false by default == 1 == False
        if pdb_id_chain != "A":
            #print(f"we are inside pdb_id_chain != A: {pdb_id_chain=}")
            pdb_original = list(pdb_id_chain)
            #we set it to A for repair.. but afterwards we swap it back!
            pdb_id_chain = "A"
    
        a = MyModel(env, alnfile=f"{pdb_code_name}.ali", knowns=full_knowns, sequence=aln_code,
                    gaps=pdb_id_target_gaps, shift=shift, chain=pdb_id_chain) #pdb_id_chain
     

        #print(f"ALIGN_CODES(1) = {a.alignment_codes[0]}")
        #a = AutoModel(env, alnfile=f"{pdb_code_name}.ali", knowns=full_knowns, sequence=aln_code)
    
        a.starting_model = 1
        a.ending_model = 1
    
        # Thorough MD optimization:
        #a.md_level = refine.slow
    
        # Repeat the whole cycle 2 times and do not stop unless obj.func. > 1E6
        #a.repeat_optimization = 2
    
        #env.libs.topology.make(aln)
        #env.libs.parameters.read(file='$(LIB)/par.lib')
        #mdl.generate_topology(aln[pdb_code_name])
        #a.generate_topology(aln[pdb_code_name])
        # Assign the average of the equivalent template coordinates to MODEL:
        #a.transfer_xyz(aln) #lets try this.
        
        # Get the remaining undefined coordinates from internal coordinates:
        #a.build(initialize_xyz=True, build_method='INTERNAL_COORDINATES')
        
        try:
            # Build the model(s)
            a.make();
            
        except Exception as e:
            print(f"An error occurred during modeling: {e}")
    
        #this worked.. now we need to clean all files that are no longer required
    
    
        try:
            #remove artifacts.
            self._remove_repair_artefacts(pdb_basep=pdb_id_target_path, pdb_code_name=pdb_code_name)
    
            #ok now we need to check what the updated start stop range is!
    
            #print(f"{max_n_terminus_found=}, {max_n_terminus_found < start_struc_query=}")
            
            #print(f"{max_c_terminus_found=}, {max_c_terminus_found > start_struc_query=}")
    
            #print(f"{start_struc_query=}")
            #print(f"{stop_struc_query=}")
            
            """
            max_n_terminus_found=-2, max_n_terminus_found < start_struc_query=True
            max_c_terminus_found=994, max_c_terminus_found > start_struc_query=True
            start_struc_query=1
            stop_struc_query=992
            keep_start=-2, keep_stop=-2
            """
    
            if pdb_original:
                #print(f"this is inside pdb_original {pdb_id_chain=}")
                #rechain back and set pdb_id_chain to original pdb chain.
                pdb_id_chain = self._rechain_back(path_to_pdb=pdb_id_target_path, pdb_original=pdb_original)
                
            #should take both chains and pdb path.. then rechain back and return back the new chain.
    
    
            keep_start = start_struc_query
            keep_stop = stop_struc_query
            
            if max_n_terminus_found and max_n_terminus_found < start_struc_query:
                keep_start = max_n_terminus_found
                    
            if max_c_terminus_found and max_c_terminus_found > stop_struc_query:
                keep_stop = max_c_terminus_found
            
            #print(f"{keep_start=}, {keep_stop=}")
            #this should select only from the start to end and excludes potentially repaired N and C termini that would only introduce more noise.
            self._select_correct_range(pdb_id_target_path, start=keep_start, stop=keep_stop)
    
    
            monomeric_chain = "".join(pdb_id_chain)
            #print(f"{monomeric_chain=}")
            #not required! ?? lets see later if it is .
            self._renumber_structure_monomeric(pdb_id_target_path, start=keep_start, chain=pdb_id_chain)
            
        except Exception as e:
            print(e)

        #if everyhing worked out we return the path!
        return pdb_id_target_path





    

    def _return_uniprot_id_from_rcsb(self, uniprot_id:str):
    
        link_path = "https://www.ebi.ac.uk/pdbe/api/mappings/uniprot"
        
        searchp = f"{link_path}/{uniprot_id}"
        #print(searchp)
        resp = get_url(searchp)
        resp = resp.json()
        
        for pdb_id, pdb_info in resp.items():
            for uniprot_id, uniprot_info in pdb_info['UniProt'].items():
                return uniprot_id


    def _get_gene_fasta(self, uniprot_id:str):
    
        #print("we are in get gene fasta")
        "this is already overworked. should work."
        #uniprot_canonical_isoform = get_uniprot_id(uniprot_id=uniprot_id)
        
        fields = "sequence"
        
        URL = f"https://rest.uniprot.org/uniprotkb/search?format=fasta&fields={fields}&query={uniprot_id}"
        resp = get_url(URL)
        resp = resp.iter_lines(decode_unicode=True)
        
        seq = ""
        
        i = 0
        for lines in resp:
            if i > 0:
                seq += lines
                #print(lines)
            i += 1
        
        #print(seq)
        return seq



    def _setup_and_prep_templates(self, template_paths)->dict:

        #print("works until here step 3:")
        start_stop_dict = defaultdict()
    
        temp_codes = []
        temp_abs_paths = []
        temp_chains = []
        
        for structure in set(template_paths):  #no duplicates here.
            #print(f"Structure: {structure} selected as template")
            temp_codes.append(structure.split("/")[-1][:-4])
            temp_abs_paths.append(structure)
            temp_chains.append(structure.split("/")[-1][-5]) #this is the chain we need.
    
        for temp_code, temp_paths, chains in zip(temp_codes, temp_abs_paths, temp_chains):
            if len(temp_code) <= 6:
                #append all start stop and shifts here.
                start_struc_temp, stop_struc_temp = get_struc_stop(temp_paths)
                start_stop_dict[temp_code] = ((start_struc_temp, stop_struc_temp, chains))
    
        return start_stop_dict

    
    def _get_struc_stop(self, path_to_pdb):
    
        parser = PDBParser()
        structure = parser.get_structure("none", path_to_pdb)
        seq_ids = [x.get_id()[1] for x in structure.get_residues()]
        seq_ids = sorted(seq_ids)
        return seq_ids[0], seq_ids[-1] #this corresponds to the last residue.


    def _remove_repair_artefacts(self, pdb_basep, pdb_code_name):
    
        pattern = f"{pdb_code_name}"
        #print(pattern)
        # Remove original_****_*.ali file
        ali_files = glob.glob(f"{pattern}.ali")
        for file in ali_files:
            os.remove(file)
        # Remove original_****_A.pdb
        old_pdb_file = f"{pattern}.pdb"
        if os.path.exists(old_pdb_file):
            os.remove(old_pdb_file)
            
        # Rename original_****_Ax.B99990001.pdb to original_****_A.pdb
        repaired_pdb_file = f"{pattern}x.B99990001.pdb"
        if os.path.exists(repaired_pdb_file):
            new_pdb_file = repaired_pdb_file.replace('x.B99990001', '')
            shutil.move(repaired_pdb_file, new_pdb_file)
    
        # Remove original_****_*x.D00000001
        d_files = glob.glob(f"{pattern}x.D00000001")
        for file in d_files:
            os.remove(file)
    
        # Remove other files
        extensions_to_remove = ['.fasta', '.ini', '.rsr', '.sch', '.V99990001']
        for ext in extensions_to_remove:
            files = glob.glob(f"{pattern}x{ext}")
            for file in files:
                os.remove(file)
                

    def _check_terminus_coverage(self, pdb_id_target_gaps, start_stop_dict):
        """
        Checks if the N-terminus and C-terminus gaps in the target structure can be covered by any of the templates.
    
        Args:
        - pdb_id_target_gaps: A list of tuples representing the gaps in the target structure.
        - start_stop_dict: A dictionary with template start and stop residue numbers and chains.
    
        Returns:
        - A tuple of booleans indicating whether suitable templates for the N-terminus and C-terminus were found.
        - Updated pdb_id_target_gaps after excluding uncovered terminus gaps.
        """
        found_N_terminus = False
        found_C_terminus = False
    
        n_terminus_range = range(pdb_id_target_gaps[0][0], pdb_id_target_gaps[0][1] + 1)
        c_terminus_range = range(pdb_id_target_gaps[-1][0], pdb_id_target_gaps[-1][1] + 1)
    
        max_n_terminus_found = None
        max_c_terminus_found = None
    
        for _, (start, stop, _) in start_stop_dict.items():
            if int(start) < n_terminus_range[-1]:
                found_N_terminus = True
                max_n_terminus_found = max(max_n_terminus_found, start) if max_n_terminus_found is not None else start
    
            if int(stop) > c_terminus_range[0]:
                found_C_terminus = True
                max_c_terminus_found = max(max_c_terminus_found, stop) if max_c_terminus_found is not None else stop
    
        # Adjust the gaps list based on the terminus coverage
        if not found_N_terminus and len(pdb_id_target_gaps) > 1:
            pdb_id_target_gaps = pdb_id_target_gaps[1:]
        elif not found_N_terminus:
            return False, False, []
    
        if not found_C_terminus and len(pdb_id_target_gaps) > 1:
            pdb_id_target_gaps = pdb_id_target_gaps[:-1]
        elif not found_C_terminus:
            return False, False, []
    
        return found_N_terminus, found_C_terminus, pdb_id_target_gaps


    def _rechain_back(self, path_to_pdb:str, pdb_original:list)-> None:

        #first check if its multichain or single chain.
        new_chains = "".join(pdb_original)
        parser = PDBParser()
        structure = parser.get_structure("default", path_to_pdb)

        for model in structure:
            for idx, chain in enumerate(model):
                if chain.id != new_chains[idx]:
                    chain.id = new_chains[idx]
    
        io = PDBIO()
        io.set_structure(structure)
        io.save(path_to_pdb)
        return pdb_original




    def _select_correct_range(self, path:str, start:int, stop:int):
    
        #sel only c_alpha
        class CAlphaOnlyInCorrectRange(Select):
            def __init__(self, start, stop, *args):
                super().__init__(*args)
                self.start = start
                self.stop = stop

            #overloaded to only accept positive residue numbering.
            def accept_residue(self, residue):      
                return 1 if residue.id[1] >= self.start and residue.id[1] <= self.stop else 0    
            
        #filelst    path
        #5ltu_A.pdb /home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN
        
        parser = PDBParser(QUIET=True)
        prot_name = f"default"
        
        #print("we are here")
        #print(fullpath)
        structure = parser.get_structure(prot_name, path)
        
        # Select C-alpha atoms and save the modified structure
        io = PDBIO()
        io.set_structure(structure)
        io.save(path, CAlphaOnlyInCorrectRange(start=start, stop=stop))



    def _renumber_structure_monomeric(self, path_to_pdb:str, start:int, chain:str):

        shiftres_location = f"{self.work_dir}/pdb_shiftres_by_chain.py"
        bash_cmd = f"python {shiftres_location} {path_to_pdb} {start-1} {chain}"
        bash_cmd_rdy = bash_cmd.split()
        
        with open(f"{path_to_pdb}_tmp", "w") as fh_tmp:
            result = run(bash_cmd_rdy, stdout=fh_tmp, stderr=PIPE, 
                 universal_newlines=True)
        
        #now replace the original one with the temp file.
        os.replace(f"{path_to_pdb}_tmp", f"{path_to_pdb}")

In [30]:
class PrincipalComponentAnalysis:

    def __init__(self, script_dir, pca_dir, work_dir,multiseq_dict=None,store_original=True):
        self.work_dir = work_dir #The work dir where our pdbs are located
        self.script_dir = script_dir #directory containing all scripts required for PCA. 
        self.store_full_atomistic = store_original #if we want to store full atomistic untouched structures
        self.atomistic_models_directory = None
        self.seqalignment_dict = multiseq_dict
        self.seqalignment_fasta = os.path.join(self.work_dir, "multiseq_fasta.fa")
        self.prepared_dir = None
        self.multi_pdb_codes = None
        self.reference_pdb = None
        self.pca_dir = pca_dir

    
    def _copy_original_pdbs(self):
        '''Helper function in order to store full atomistic PDBs untouched
        and uncutted for downstream analysis like mutational mapping
        We simply copy all files from the working directory to another directory called:
        --------------------------------------------------------------------------------
        work_dir/full_pdb_models/ 
        --------------------------------------------------------------------------------
        There we will find all pdbs that are present in the work dir before PCA run.
        '''
        if self.store_full_atomistic:
            # List all PDB files in the working directory
            pdbs = [f for f in os.listdir(self.work_dir) if os.path.isfile(os.path.join(self.work_dir, f)) and f.endswith(".pdb")]
            # Define the directory where PDB files will be stored
            store_dir = os.path.join(self.work_dir, "full_pdb_models")
            # Create the directory if it doesn't exist
            if not os.path.exists(store_dir):
                os.mkdir(store_dir)
            # Copy each PDB file to the storage directory
            for pdb in pdbs:
                src = os.path.join(self.work_dir, pdb)  # Source file path
                dst = os.path.join(store_dir, pdb)  # Destination file path
                shutil.copy(src, dst)  # Copy file to new dir
            self.atomistic_models_directory = store_dir


    
    def prepare_ensemble(self, strict_cutting="strict"):
        '''
        This function prepares the ensemble of PDBs and does the following steps:
        - Removal of hetero atoms like H2O and Ligands.
        - Skeletonization of the models to strip down all atoms except CAs which will be used for PCA (full atomistic models contain too much
        noise to be used in PCA due to side chain fluctations while CAs should not fluctuate much besides for conformers which is the aim
        of finding in this PCA)
        
        - Cutting based on a structure based alignment.
        Here we offer two options: strict cutting or intelligent cutting.
        
            - strict cutting: We directly take the ensemble, check for gaps in aligned positions 
            and REMOVE the atoms of a column IF there is a gap in 1 position.

            - intelligent cutting: We first try to find structures in the ensemble that introduce many gaps.
            We proceed by removal of those structures, rerunning the alignment and then continue with normal strict cutting.
            This can significantly improve the number of retained positions in the final PCA, but comes at the cost of dropping potentially
            relevant structures.
        
        '''

        #1 we select CA only. / we obtain info about potential non canonical aas.
        pdbs = [os.path.join(self.work_dir,f) for f in os.listdir(self.work_dir) if os.path.isfile(os.path.join(self.work_dir, f)) and f.endswith(".pdb")]

        with ProcessPoolExecutor() as executor:
            remove_het_results = list(executor.map(self._select_c_alpha_and_check_non_canonical, pdbs))
        
        #2 remove non canonical aas.
        for non_canonical, pdb_path in remove_het_results:
            if non_canonical:
                for keys, vals in non_canonical.items():
                    self._mutate_non_standard_aa(pdb_path, non_standard_residue=vals[0], residue=keys, chain=vals[1])

        #now we leverage the seq alignment from USAlign.
        if cutting == "strict":
            #print("we move into strict cutting")
            residues_to_keep = self._hard_trimmer(self.seqalignment_fasta)
            save_dir = os.path.join(self.work_dir, "trimmed_strucs")
            print(f"{save_dir=}")
            if not os.path.exists(save_dir):
                os.mkdir(save_dir)
            #this will be used to run PCA.
            self.prepared_dir = save_dir
            self._select_proper_residues_after_trim(residues_to_keep, save_dir)

        elif cutting == "intelligent":
            #these strucs need to be removed.
            strucs_to_remove = self._soft_trimmer(self.seqalignment_fasta)
            #and then we rerun usalign.
            #we remove the strucs that are found to introduce many gaps.
            self._hard_trimmer(self.seqalignment_fasta)
            
    
    def _soft_trimmer(self, result_fasta:str):
        #maybe better idea is instead of removing so harshly to set a treshold. if its a long prot seq 
        #we dont want to kick out stuff that introduces 5 gaps e.g serca because mean is already 2... so very low..
        #in this case we are perfectly fine in introducing a gap of up to 10% of total len
        gap_counter = defaultdict()
        ali = pytrimal.Alignment.load(result_fasta)
        
        trimmer = pytrimal.AutomaticTrimmer("gappyout")
        trimmed = trimmer.trim(ali)
        
        for name, seq in zip(trimmed.names, trimmed.sequences):
            structure_name = name.decode().rjust(6)
            gap_count = seq.count("-")
            #print(seq)
            gap_counter[structure_name] = gap_count

        gap_values = list(gap_counter.values())
        mean_gap_count = statistics.mean(gap_values)
        median_gap_count = statistics.median(gap_values)
    
        print(mean_gap_count)
        print(median_gap_count)
        high_gap_count_sequence_ids = [seq_id for seq_id, gap_count in gap_counter.items() if gap_count > mean_gap_count]
    
        print(high_gap_count_sequence_ids)
        return high_gap_count_sequence_ids
    

    def _hard_trimmer(self, result_fasta:str):

        keep_res_dict = defaultdict()
        ali = pytrimal.Alignment.load(result_fasta)
        #trimm all gaps unless this leaves 10% of the original alignment.
        trimmer = pytrimal.ManualTrimmer(gap_threshold=1) #changed to 80% now as test.
        trimmed = trimmer.trim(ali)
        for name, seq in zip(trimmed.names, trimmed.sequences):
            keep_res_dict[name.decode().rjust(6)] = seq

        return keep_res_dict

    def _select_proper_residues_after_trim(self, keep_residue_dict:dict, save_dir:dir):
        
        for pdb, seq in keep_residue_dict.items():
            #seq in keep_residue_dict is extracted from hard trimmer.
        
            full_p = os.path.join(self.work_dir, pdb)
            result_seq = self._grab_sequence_from_struc(full_p)
            positions_to_keep = self._find_positions(result_seq[0], seq)
            #print(f"this is seq to keep: {seq}, this is full_seq from structure: {result_seq}")
            #print(f"this is our savepath for cutted struc: {full_p}")
            self._select_trimmed_positions(path_to_pdb=full_p, list_to_keep=positions_to_keep, save_dir=save_dir)
            

    def _select_trimmed_positions(self, path_to_pdb:str, list_to_keep:str, save_dir):
        #sel only c_alpha
        class CAlphaTrimmedPositions(Select):
            def __init__(self, list_to_keep_shifts, *args):
                super().__init__(*args)
                self.list_to_keep_shifts = list_to_keep_shifts
                
            #overload accept_residue inherited from Select with this conditional return
            def accept_atom(self, atom):
                return 1 if atom.id == "CA" else 0
            
            #overloaded to only accept positive residue numbering.
            def accept_residue(self, residue):      
                return 1 if residue.id[1] in self.list_to_keep_shifts else 0    

        #print(f"{path_to_pdb=}, {list_to_keep=}")
        #lets make a new dir where we store the trimmed strucs.
        pdb_name = os.path.basename(path_to_pdb)
        new_location = os.path.join(save_dir, pdb_name) #here we save
        parser = PDBParser(QUIET=True)
        prot_name = f"default"
        structure = parser.get_structure(prot_name, path_to_pdb)
        shift = [x.get_id()[1] for x in structure.get_residues()][0] #first residue number = shift
        list_to_keep_shifted = [x+shift-1 for x in list_to_keep] #correct for shift. lets try with shift
        #print(f"this is structure: {path_to_pdb} for list to keep shifted: {list_to_keep_shifted}")
        # Select C-alpha atoms and save the modified structure
        io = PDBIO()
        io.set_structure(structure)
        io.save(new_location, CAlphaTrimmedPositions(list_to_keep_shifts=list_to_keep_shifted))
    

    def _find_positions(self, full_sequence, partial_sequence):
        i = 0  # Index for full_sequence
        j = 0  # Index for partial_sequence
        found_positions = []
    
        while i < len(full_sequence) and j < len(partial_sequence):
            if partial_sequence[j] == '-':  # Skip gap positions in partial_sequence
                j += 1
            elif partial_sequence[j] == full_sequence[i]:  # Match found
                found_positions.append(i)
                i += 1
                j += 1
            else:
                i += 1  # Advance in full_sequence if no match
    
        found_pos = [x + 1 for x in found_positions]  # Adjust for 1-based indexing
        return found_pos

    def _grab_sequence_from_struc(self, pdb):

        lst =  [('VAL',"V"), ('ILE',"I"), ('LEU',"L"), ('GLU',"E"), ('GLN',"Q"),
                        ('ASP',"D"), ('ASN',"N"), ('HIS',"H"), ('TRP',"W"), ('PHE',"F"), ('TYR',"Y"), 
                        ('ARG',"R"), ('LYS',"K"), ('SER',"S"), ('THR',"T"), ('MET',"M"), ('ALA',"A"), 
                        ('GLY',"G"), ('PRO',"P"), ('CYS',"C")]
        
        canonical_aas = defaultdict(lambda: "X", lst)
        parser = PDBParser(QUIET=True)
        pdb_name = os.path.basename(pdb)
        prot_name = f"default"
        structure = parser.get_structure(prot_name, pdb)
        struc_seq = [canonical_aas[x.get_resname()] for x in structure.get_residues()]
        struc_seq = "".join(struc_seq)
    
        return (struc_seq, pdb_name)
    
    
    def run_PCA(self):
        if not self.prepared_dir:
            return
        if self.store_full_atomistic:
            self._copy_original_pdbs()
            
        self.multi_pdb_codes, self.reference_pdb = self._create_multi_pdb()
        
        print(f"{self.multi_pdb_codes=}, {self.reference_pdb=}")
        if self.multi_pdb_codes and self.reference_pdb:
            #now lets run it.
            print("we shuffle and go")
            self._shuffle_exe_and_run_pca()
        
    
    def _create_multi_pdb(self, reference=None):
        
        #this function acts as an helper function in pca_laura_pipeline_1
        pdb_files = [f for f in os.listdir(self.prepared_dir) if f.endswith(".pdb")]
        #lets do first a check of length so we only keep strucs that are pca-able.
        count_dict = Counter()
        for file in pdb_files:
            location = os.path.join(self.prepared_dir, file)
            prot_name = file[:-4]
            try:
                structure = PDBParser(QUIET=True).get_structure(prot_name, location)
                struc_len = len([x for x in structure.get_residues()])
                count_dict[struc_len] += 1
                
            except Exception as e:
                print(e)
            
        most_common_lengths = count_dict.most_common()
        most_common_length = most_common_lengths[0][0]
        print(most_common_length)
        ms = Structure.Structure("master")
        i = 0
        
        list_of_pdb_codes = []
        for file in pdb_files:
            location = os.path.join(self.prepared_dir, file)
            prot_name = file[:-4]
    
            try:
                structure = PDBParser(QUIET=True).get_structure(prot_name, location)
                struc_len = len([x for x in structure.get_residues()])
    
                if struc_len != most_common_length:
                    print(f"This structure has wrong amount of residues: {location=} {struc_len=} instead of {most_common_length=}")
                    #dont proceed in this case.
                    continue    
            except Exception as e:
                print(f"Error parsing {file}: {e}")
                continue
            
            list_of_pdb_codes.append(file)
            for model in structure:
                new_model = model.copy()
                new_model.id = i
                new_model.serial_num = i + 1
    
                # Iterate through atoms and set B-factor to 0 because this causes issues with pcatoolS
                for chain in new_model:
                    for residue in chain:
                        for atom in residue:
                            atom.set_bfactor(0)
    
                i += 1
                ms.add(new_model)
    
        pdb_io = PDBIO()
        pdb_io.set_structure(ms)
        pdb_io.save(f"{self.prepared_dir}/multi_ensemble.pdb")

        if reference:
            path_of_ref = None
        else:
            path_of_ref = os.path.join(self.prepared_dir, list_of_pdb_codes[0])

        
        return list_of_pdb_codes, path_of_ref 


    def _move_executables(self, basepath, work_dir):
        #this helper function is required for domenico and laura stuff to work.
        os.chdir(work_dir) #i dislike this solution but its fine
        try:
            moved_files_new_location = []
            files_to_move = [f for f in os.listdir(basepath) if os.path.isfile(os.path.join(basepath, f))]
            for file in files_to_move:
                shutil.copy(os.path.join(basepath, file), work_dir)
                #we want to delete them afterwards.
                moved_files_new_location.append(os.path.join(basepath, file), work_dir)
                return moved_files_new_location
        except Exception as error:
            print(error)
            return None

    def _shuffle_exe_and_run_pca(self, protein="test"):
        file_lst_to_be_removed = self._move_executables(self.pca_dir, self.prepared_dir)
        template = os.path.basename(self.reference_pdb) # this is the template pdb file name
        
        #_move_executables("/home/micnag/bioinformatics/rcsb_retrieved_pdbs/PCA_pipeline", work_dir)
        # Part 1: PCA
        bash_cmd1 = f"./getpca.sh multi_ensemble.pdb {template} refpdb_ref"
        print(f"this is bashcmd 1: {bash_cmd1}")
        self._run_command(bash_cmd1, "Part 1")
        # Part 2: Convert evec files
        bash_cmd2 = f"./convert.sh refpdb_ref_pca.evec"
        print(f"this is bashcmd 2: {bash_cmd2}")
        self._run_command(bash_cmd2, "Part 2")
        # Part 3: Projections
        bash_cmd3 = f"./getproj.sh {template} refpdb_ref_pca.evec {protein} multi_ensemble.pdb"
        print(f"this is bashcmd 3: {bash_cmd3}")
        self._run_command(bash_cmd3, "Part 3")
        # NOW lets remove the exe etc again. 
        for filep in file_lst_to_be_removed:
            try:
                os.remove(filep)
            except Exception as e:
                print(e)

    def _run_command(self, bash_cmd, part_name):
        try:
            result = run(bash_cmd.split(), stdout=PIPE, stderr=PIPE, universal_newlines=True)
            print(f"{part_name} output:\n{result.stdout}")
            print(f"{part_name} errors:\n{result.stderr}")
        except Exception:
            print(f"Script did not work. Parameters: {bash_cmd.split()}")

    def plot_PCA(self, name:str, pdb_codes:list, 
                  save_df=True, save_img=True, **kwargs):

        """kwargs can be used to modify the plot: e.g pass
        layout_config={'width': 1000, 'height': 700}, trace_config={'marker_size': 12}
        to the function call beforehand."""
        
        proj_dict = defaultdict()
    
        save_pdb_file_codes = os.path.join(self.work_dir, "pdb_codes_PCA.txt")
        with open(save_pdb_file_codes, "w") as pdb_fh:
            for pdb_code in pdb_codes:
                pdb_fh.write(pdb_code)
                pdb_fh.write("\n")
    
        var_list = []
    
        #they are already sorted.
        with open(f"{input_dir}/{protein}_evec3.dat", "r") as var_readin:
            for lines in var_readin:
                line = lines.split()
                if len(line) == 2:
                    var_list.append(float(line[1]))

            expl_var = var_list / np.sum(var_list)

        mode_proj_path = os.path.join(self.work_dir, name)
        with open(f"{mode_proj_path}.mode_12.proj", "r") as res_proj:
            for i, lines in enumerate(res_proj):
                # Split the data into lines
                lines = lines.strip().split('\n')
                # Extract the last two columns from each line
                res_lines = [(line.split()[-2], line.split()[-1]) for line in lines]
                PC1 = float(res_lines[0][0])
                PC2 = float(res_lines[0][1])
                proj_dict[str(i)] = (PC1, PC2)
            
        PC_1 = []
        PC_2 = []
        for keys, vals in proj_dict.items():
            #print(vals)
            PC_1.append(vals[0])
            PC_2.append(vals[1])
    
        plot_df = pd.DataFrame(columns=["PC1", "PC2", "labels"])
        plot_df["PC1"] = PC_1
        plot_df["PC2"] = PC_2
        plot_df["labels"] = pdb_codes

        if save_df:
            save_location_df = os.path.join(self.work_dir, f"{protein}_pca_df.csv")
            plot_df.to_csv(save_location_df)

        # Default configurations for plot
        default_layout_config = {
        'plot_bgcolor': 'lavender',
        'paper_bgcolor': 'white',
        'width': 800,
        'height': 600,
        'xaxis_title': f'PC 1 {expl_var[0]*100:.2f}%',
        'yaxis_title': f'PC 2 {expl_var[1]*100:.2f}%',
        'legend_title_text': 'Clusters',
        'showlegend': True,
        'title': {
            'text': f'PCA of {name}: {len(pdb_codes)} structures',
            'x': 0.5,
            'font': {'size': 30}
            }
        }

        # Update default configurations with any kwargs provided
        default_layout_config.update(kwargs.get('layout_config', {}))

        fig = px.scatter(plot_df, x= "PC1",y= "PC2",
                hover_data=["labels"],labels={"labels": "labels"},
                custom_data = ["labels"])

        fig.update_traces(marker_size=10,
                      hovertemplate="<b>RCSB: %{customdata[0]}</b>",
                      hoverlabel=dict(font_size=40),  # Set the hover label font size
                      mode="markers+text", selector=dict(type='scatter'),
                      **kwargs.get('trace_config', {}))  # Apply trace configurations from kwargs
    
        fig.update_layout(**default_layout_config)
    
        if save_image:
            img_save_path = os.path.join(self.work_dir, f"{name}_PC_plot.png")
            pio.write_image(fig, img_save_path, format="png")
    
        return fig.show()
    
    def _select_c_alpha_and_check_non_canonical(self, pdb_path):
        #sel only c_alpha
        class CAlphaOnlyInCorrectRange(Select):
            #overload accept_residue inherited from Select with this conditional return to save only CA
            def accept_atom(self, atom):
                return 1 if atom.id == "CA" else 0
            def accept_residue(self, residue):
                return 1 if residue.id[0] == " " else 0


        non_canonical_aas = defaultdict()

        canonical_aas = {'VAL', 'ILE', 'LEU', 'GLU', 'GLN',
                     'ASP', 'ASN', 'HIS', 'TRP', 'PHE', 'TYR',
                     'ARG', 'LYS', 'SER', 'THR', 'MET', 'ALA',
                     'GLY', 'PRO', 'CYS'}
        
        parser = PDBParser(QUIET=True)
        structure = parser.get_structure('default', pdb_path)

        for model in structure:
            for chain in model:
                for residue in chain:
                    if residue.id[0] == " ": 
                        curr_res = residue.get_resname()
                        curr_pos = residue.get_id()[1]

                        if curr_res not in canonical_aas:
                            non_canonical_aas[curr_pos] = (curr_res, chain.get_id())

            
        # Select C-alpha atoms and save the modified structure
        io = PDBIO()
        io.set_structure(structure)
        #print(f"we save {pdb_path=}")
        io.save(pdb_path, CAlphaOnlyInCorrectRange())
        return non_canonical_aas, pdb_path


    def _count_unique_values_in_dict(self, input_dict):
        start_counts = Counter(val[0] for val in input_dict.values())
        stop_counts = Counter(val[1] for val in input_dict.values())
        return start_counts, stop_counts

    
    def _mutate_non_standard_aa(self, path_to_pdb):
        """Mutates a non-standard amino acid in a PDB file."""
        
        path_to_script = f"{self.script_dir}/pdb_mutate.py"
        path_to_error_script = f"{self.script_dir}/pdb_delresname.py"
        path_to_tidy = f"{self.script_dir}/pdb_tidy.py"
        input_file = f"{path_to_pdb}_new" #temp name to not overwrite files we currently read from
        try:
            with open(input_file, "w") as pdb_out:
                bash_code = f"python {path_to_script} {path_to_pdb} {chain} {residue} {non_standard_residue} ALA"
                run(bash_code.split(), stdout=pdb_out, stderr=PIPE, universal_newlines=True)
            resulting_success = True
    
        except Exception as error:
            print(error)
            print("Attempting to delete non-standard residue...")
            
            with open(input_file, "w") as pdb_out:
                bash_code = f"python {path_to_error_script} -{non_standard_residue} {path_to_pdb}"
                run(bash_code.split(), stdout=pdb_out, stderr=PIPE, universal_newlines=True)
    
        #now we tidy the file to adhere to the most common pdb standard
        try:
            with open(path_to_pdb, "w") as pdb_out:
                bash_code = f"python {path_to_tidy} {input_file}"
                run(bash_code.split(), stdout=pdb_out, stderr=PIPE, universal_newlines=True)
            #print("Cleaned the file. Outfile is at", path_to_pdb)
            
        except Exception as error:
            print(error)
    
        #we dont need the intermediate file that was only created to prevent read/write from same file.
        try:
            os.remove(input_file)  # Remove intermediate file
        except Exception as error:
            print(error)
            
        return resulting_success
        
    def _get_struc_stop(self, path_to_pdb):

        parser = PDBParser()
        structure = parser.get_structure("none", path_to_pdb)
        seq_ids = [x.get_id()[1] for x in structure.get_residues()]
        seq_ids = sorted(seq_ids)
        return seq_ids[0], seq_ids[-1] #this corresponds to the last residue.




We already have a pdb_download file at /home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/pdb_list.csv


In [32]:
#PdbEnsemble_meta = Downloader.meta_dict
#PdbEnsemble_chains_seq_id = Downloader.chain_seqid_dict

#Downloader.conservation(uniprot_id="Q9NZJ9")
#shifts = PdbEnsemble.parallel_shift_calculation()
#Downloader.conservation_df

In [33]:
PDB_Cleaner = PDBCleaning(work_dir=work_dir, meta_dict=PdbEnsemble_meta, chain_seq_dict=PdbEnsemble_chains_seq_id)
PDB_Cleaner.setup_cutoff(cutoff=3, apply_filter=True)  #apply filter to only include structures that are of good quality
#PDB_Cleaner.parallel_shift_calculation()  # compute shift for each structure
#PDB_Cleaner.parallel_renumbering()  # renumber based on shifts.
#PDB_cleaned_ensemble.chain_dict

#PDB_Cleaner.filtered_structures
struct = PDB_Cleaner.filtered_structures
#struct = PDB_Cleaner.get_unfiltered_strucs()
#chain_seqid = PDB_Cleaner.chain_seq_dict

We have no meta dict to implement a cutoff


In [34]:
print(struct) #get_unfiltered_strucs() misses out on all files with multi chains
#['3i7v_A', '6woh_A', '6wog_A', '4hfq_AB', '6wob_A', '3h95_AB', '7aui_A', '6woe_A', '6pcl_A', '2q9p_A', '7aul_A', '6wod_A', '6wo8_A', '6woa_A', '6wo9_A', '6wof_A', '3i7u_ABCD', '6pck_A', '7aup_A', '7auk_A', '7aus_A', '7aut_A', '2fvv_A', '6wo7_A', '7nnj_A', '7tn4_A', '6woi_A', '7aun_A', '6woc_A', '3mcf_A', '7aur_A']

['3i7v', '6woh', '6wog', '4hfq', '6wob', '3h95', '5ltu', '7aui', '6woe', '6pcl', '2q9p', '7aul', '6wod', '6wo8', '6woa', '6wo9', '2duk', '6wof', '3i7u', '7auj', '7auo', '6pck', '7auq', '7aup', '7auk', '7aus', '7aum', '7aut', '2fvv', '6wo7', '7auu', '7nnj', '7tn4', '6woi', '7aun', '6woc', '3mcf', '7aur']


In [35]:
#print(chain_seqid)

In [36]:
#PDB_Builder = PDBBuilder(work_dir=work_dir, structures=struct, remove_intermediates=True) #structures that are filtered
#PDB_Builder.build_assembly()
#oligo_dict = PDB_Builder.oligodict

In [37]:
#print(oligo_dict)

In [38]:
#Preper = PDBEnsemblePrep(work_dir=work_dir, oligo_dict=oligo_dict, chain_seq_dict=chain_seqid, main_prot_seq=None)
#Preper.create_domain_boundaries()
#Preper.get_oligostates(num_most_common_oligostates=3)
#Preper.process_templates()
#top_templates = Preper.top_templates

In [39]:
#USAligner = USAlign(work_dir=work_dir, structure_seqid_dict=Preper.templates_for_oligos, 
#                    template_min_identity=0.1,
#                   num_top_clusters_per_range=4)

#USAligner.USAlign_run()
#USAligner.filter_results(tm_cutoff=0.8,rmsd_min_cutoff=0.2, log_file=True)
#USAligner.setup_oligo_directories()
#result_dict = USAligner.result_dict
#USAligner.run_multiseq_alignment(directory="/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/monomer/1-203")

In [40]:
#Repairstation = ModellerRepairEnsemble(ensemble_dict=result_dict, work_dir=work_dir)
#print(Repairstation.repair_ensemble)
#print(Repairstation.monomers)
#Repairstation.repair_structures()

In [42]:
#script_dir = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/scripts"

#work_dir = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/monomer/1-203"

#fasta = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/monomer/1-203/multiseq_fasta.fa"

#pca_dir = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/Test_OOP_pipeline/pca_part"

#PCA_object = PrincipalComponentAnalysis(work_dir=work_dir, script_dir=script_dir, 
#                                        pca_dir=pca_dir,store_original=True)
#PCA_object.run_PCA(work_dir)
#PCA_object.prepare_ensemble()
#PCA_object.run_PCA()  # works but is crap because of forced path locations for executables.

In [ ]:
#PCA_object.multi_pdb_codes